<a href="https://colab.research.google.com/github/marcusapel/ecletp/blob/main/ecim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OSDU Hands-on Lab Environment Setup

This notebook provides hands-on exercises for interacting with the OSDU Data Platform using Python and REST APIs.

**Target Audience:** Users familiar with basic data concepts but not necessarily developers.

**Goal:** To demonstrate fundamental interactions with OSDU services like Storage, Search, Legal, and Domain Data Management Services (DDMS).

**Structure:**
1.  **Initial Setup:** Dependency installation and authentication.
2.  **Lab Exercises:** Step-by-step instructions for specific tasks.

**How to Use:**
*   Read the Markdown cells for explanations.
*   Run the Code cells sequentially by clicking the 'Play' button or using `Shift+Enter`.
*   Observe the output generated by each code cell.

## 1. Initial Setup

### 1.1 Install Dependencies

This command installs the necessary Python libraries required for the labs.

In [1]:
%pip install --quiet --extra-index-url https://community.opengroup.org/api/v4/projects/148/packages/pypi/simple requests requests-oauthlib osdu-api==0.28.0 tenacity dotenv ipython pandas pyarrow matplotlib plotly numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 657.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00


### 1.2 Configure Connection and Authentication

This section configures the connection details for your OSDU instance and handles authentication. It attempts to load configuration from a `.env` file first, and falls back to hardcoded variables if the file is not found or incomplete.

**Using a `.env` file (Recommended):**

1.  Create a file named `.env` in the same directory as this notebook.
1.  Copy the content from .env.sample and fill out the variables.
1.  Add the following environment variables to the file, replacing the example values with your actual OSDU instance details:
1.  Save the `.env` file.

*(On Windows, you can create this file using Notepad or any text editor. Make sure it's saved as `.env` and not `.env.txt`)*

| Variable | Description | Example |
|-----|-------------|---------|
| osdu_endpoint | This is the base endpoint of you OSDU instance (before /api/) | https://osdu.osdu-bootcamp.com/api/config/v1/postman-environment |
| osdu_data_partition_id | This is the data partition ID you want to connect to | osdu |
| osdu_group_domain | This is the Entitlements Domain (or Group Domain) (users@{dataPartitionId}.<span style="color:green">{osduGroupDomain}</span>) | company.com |
| token_endpoint | OAuth2.0 *token* endpoint (ends with /token) | https://idpdomain.com/oauth2/v2.0/token |
| client_id | *optional* App registration/client ID | osdu-admin |
| client_secret | *optional* App registration/client secret | YTOonDBJfSGxoqje |
| scope | *optional* Permissions that the token will have | aaaaaaaa-bbbb-cccc-dddd-eeeeeeeeeeee/.default |
| refresh_token | *optional* User Refresh Token if client credentials are not supported | RjY2NjM5NzA2OWJjuE7c... |

**Fallback (Hardcoded):**

If the `.env` file is not found or you prefer not to use one, you can directly edit the `initialize_from_local_variables` function within the code cell below to set your configuration details. **However, avoid committing sensitive information like secrets directly into notebooks.**

### 1.3 Generate Personal Identifier

This creates a unique suffix (`user_id`) that will be appended to the names of resources created during this lab session. This helps prevent naming conflicts if multiple users run the labs simultaneously against the same OSDU instance.

In [2]:
import random

# Generate a random user_id with 4 digits
user_id = f"{random.randint(1000, 9999)}"
print(f"Generated user_id for this session: {user_id}")
print("This ID will be used to create unique resource names.")

Generated user_id for this session: 7073
This ID will be used to create unique resource names.


In [3]:
import time
import json
import os
import requests
import pandas as pd
import urllib
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from dotenv import load_dotenv
from IPython.display import Markdown, display, HTML

# --- Configuration Loading ---
dotenv_file = ".env"

# Initialize global variables to None
osdu_endpoint = None
osdu_data_partition_id = None
osdu_group_domain = None
token_endpoint = None
client_id = None
client_secret = None
scope = None
refresh_token = None
headers = None
parquet_headers = None
entitlements_domain = None

def initialize_from_local_variables():
    """Sets config from hardcoded values if .env fails. **Modify with caution**."""
    global osdu_endpoint, osdu_data_partition_id, osdu_group_domain, token_endpoint, client_id, client_secret, scope, refresh_token

    # --- OSDU instance configuration (MODIFY HERE IF NOT USING .env) ---
    local_osdu_endpoint = "https://osdu.osdu-bootcamp.com"
    local_osdu_data_partition_id = "osdu"
    local_osdu_group_domain = "group"

    # --- OAuth2 configuration (MODIFY HERE IF NOT USING .env) ---
    local_token_endpoint = "https://keycloak.osdu-bootcamp.com/realms/osdu/protocol/openid-connect/token"  # e.g., "https://your-idp.com/oauth2/v2.0/token"
    local_client_id = "osdu-admin"
    local_client_secret = "YTOonDBJfSGxoqje"
    local_scope = []
    local_refresh_token = "" # Required only if client_credentials grant is not supported
    # --- End of modification section ---

    if local_osdu_endpoint and local_osdu_data_partition_id: # Basic check if values were entered
        osdu_endpoint = local_osdu_endpoint
        osdu_data_partition_id = local_osdu_data_partition_id
        osdu_group_domain = local_osdu_group_domain
        token_endpoint = local_token_endpoint
        client_id = local_client_id
        client_secret = local_client_secret
        scope = local_scope
        refresh_token = local_refresh_token
        print(f"Configuration loaded from local variables within the notebook.")
        return True
    else:
        print("Local variables seem empty, skipping.")
        return False

def initialize_from_env_file(file_path):
    """Loads config from a .env file."""
    global osdu_endpoint, osdu_data_partition_id, osdu_group_domain, token_endpoint, client_id, client_secret, scope, refresh_token
    try:
        if load_dotenv(file_path, override=True):
            print(f"Attempting to load configuration from '{file_path}'...")
            osdu_endpoint = os.getenv("OSDU_ENDPOINT")
            osdu_data_partition_id = os.getenv("OSDU_DATA_PARTITION_ID")
            osdu_group_domain = os.getenv("OSDU_GROUP_DOMAIN")
            token_endpoint = os.getenv("TOKEN_ENDPOINT")
            client_id = os.getenv("CLIENT_ID")
            client_secret = os.getenv("CLIENT_SECRET")
            scope_str = os.getenv("SCOPE")
            scope = list(map(lambda x: x.strip(), scope_str.split(","))) if scope_str else None
            refresh_token = os.getenv("REFRESH_TOKEN")

            # Basic validation
            if all([osdu_endpoint, osdu_data_partition_id, token_endpoint, client_id, (client_secret or refresh_token)]):
                 print(f"Configuration successfully loaded from '{file_path}'.")
                 return True
            else:
                print(f"Configuration file '{file_path}' is missing one or more required variables.")
                return False
        else:
            print(f"Configuration file '{file_path}' not found or empty.")
            return False
    except Exception as e:
        print(f"Error loading configuration from '{file_path}': {e}")
        return False

# --- Token Management ---
_token_context = {
    "token": None,
    "expires_at": 0
}

def fetch_new_token():
    """Fetches a new OAuth2 token using client credentials or refresh token."""
    if not all([token_endpoint, client_id]):
        raise ValueError("Token Endpoint and Client ID must be configured.")

    token = None
    oauth = None # Initialize oauth session

    # Try client credentials flow first (requires client_secret)
    if client_secret:
        try:
            print("Attempting token fetch using client credentials...")
            client = BackendApplicationClient(client_id=client_id) #, scope=scope)
            oauth = OAuth2Session(client=client)
            token = oauth.fetch_token(
                token_url=token_endpoint,
                client_id=client_id,
                client_secret=client_secret #,
                # scope=scope
            )
            print("Token fetched successfully via client credentials.")
        except Exception as e:
            print(f"Client credentials flow failed: {e}. Checking for refresh token.")
            token = None # Ensure token is None if this flow fails

    # If client credentials failed or no secret provided, try refresh token flow (requires refresh_token)
    if token is None and refresh_token:
        try:
            print("Attempting token refresh...")
            # Need a session object even for refresh if not created above
            if oauth is None:
                 client = BackendApplicationClient(client_id=client_id, scope=scope)
                 oauth = OAuth2Session(client=client)

            token = oauth.refresh_token(
                token_url=token_endpoint,
                client_id=client_id, # Some IDPs might still require client_id/secret for refresh
                client_secret=client_secret,
                refresh_token=refresh_token
            )
            print(f"Token refreshed successfully.")
        except Exception as e:
            print(f"Refresh token flow failed: {e}")
            token = None

    # If both flows failed
    if token is None:
         raise ConnectionError("Failed to obtain authentication token using both client credentials and refresh token methods.")

    _token_context["token"] = token["access_token"]
    # Use 'expires_at' if provided, otherwise calculate from 'expires_in'
    _token_context["expires_at"] = token.get("expires_at", time.time() + token.get("expires_in", 3600) - 60) # 60s buffer

def get_token():
    """Gets the current valid token, fetching a new one if needed."""
    if _token_context["token"] is None or _token_context["expires_at"] <= time.time():
        print("Token is expired or not available. Fetching new token...")
        fetch_new_token()
    # else:
        # print("Using existing token.") # Optional: for debugging
    return _token_context["token"]

# --- UI Helpers ---
def printmd(string):
    """Displays text as Markdown."""
    display(Markdown(string))

def display_banner(text, color):
    """Displays a colored banner."""
    background_color = 'lightgreen' if color == 'green' else 'lightcoral'
    text_color = 'black'
    html_banner = f'''
        <div style="
            background-color: {background_color};
            color: {text_color};
            font-size: 16pt;
            font-weight: bold;
            text-align: center;
            padding: 15px;
            border-radius: 8px;
            margin: 15px 0;
        ">
            {text}
        </div>
    '''
    display(HTML(html_banner))

# --- Initialization and Validation ---
if __name__ == "__main__":
    config_loaded = False
    # Try loading from .env file first
    if initialize_from_env_file(dotenv_file):
        config_loaded = True
    else:
        # Fallback to local variables if .env loading fails
        print("Falling back to loading configuration from local variables...")
        if initialize_from_local_variables():
            config_loaded = True
        else:
             print("Failed to load configuration from local variables as well.")

    if not config_loaded:
        error_message = "CONFIGURATION FAILED: Could not load connection details from .env file or local variables. Please check the setup instructions."
        display_banner(error_message, color='red')
        # Stop execution if config failed
        raise ValueError(error_message)
    else:
        try:
            print("Configuration loaded. Attempting authentication...")
            token = get_token() # This will trigger fetch_new_token

            # Construct Entitlements Domain (handle case where group domain might be missing)
            if osdu_group_domain:
                 entitlements_domain = f"{osdu_data_partition_id}.{osdu_group_domain}"
            else:
                 entitlements_domain = osdu_data_partition_id # Or handle as an error if required
                 print(f"Warning: OSDU_GROUP_DOMAIN is not set. Entitlements might not work as expected.")

            # Prepare standard request headers
            headers = {
                "data-partition-id": osdu_data_partition_id,
                "content-type": "application/json",
                "accept": "application/json",
                "authorization": f"Bearer {get_token()}", # Use get_token() to ensure it's valid
            }

            # Prepare headers for Parquet uploads
            parquet_headers = {
                "data-partition-id": osdu_data_partition_id,
                "content-type": "application/x-parquet",
                "accept": "application/json", # Usually expect JSON response even for binary upload
                "authorization": f"Bearer {get_token()}"
            }

            # Test connection with a simple Search API call
            print("Testing connection to OSDU Search service...")
            test_url = f"{osdu_endpoint}/api/search/v2/query"
            test_payload = json.dumps({
                "kind": "*:*:*:*",
                "query": "*",
                "limit": 1
            })

            response = requests.post(test_url, headers=headers, data=test_payload)
            response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

            if response.status_code == 200:
                display_banner("AUTHENTICATION SUCCESSFUL<br><span style='font-size: 12pt;'>Connected to OSDU instance.</span>", color='green')
                print(f"OSDU Endpoint: {osdu_endpoint}")
                print(f"Data Partition ID: {osdu_data_partition_id}")
            else:
                 # This case might not be reached due to raise_for_status, but included for completeness
                 error_details = f"<span style='font-size: 12pt;'>Received status code {response.status_code}.</span>"
                 display_banner(f"AUTHENTICATION FAILED<br>{error_details}", color='red')
                 print(f"Response: {response.text}")

        except Exception as e:
            error_details = f"<span style='font-size: 12pt;'>Exception: {str(e)}</span>"
            display_banner(f"AUTHENTICATION FAILED<br>{error_details}", color='red')
            # Optionally print more details for debugging
            # import traceback
            # traceback.print_exc()

Configuration file '.env' not found or empty.
Falling back to loading configuration from local variables...
Configuration loaded from local variables within the notebook.
Configuration loaded. Attempting authentication...
Token is expired or not available. Fetching new token...
Attempting token fetch using client credentials...
Client credentials flow failed: HTTPSConnectionPool(host='keycloak.osdu-bootcamp.com', port=443): Max retries exceeded with url: /realms/osdu/protocol/openid-connect/token (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2751ea720>, 'Connection to keycloak.osdu-bootcamp.com timed out. (connect timeout=None)')). Checking for refresh token.


###1.4 Google Colab Functions

In [4]:
### 1.3 Google Colab Functions

# get a file from the bucket
from google.cloud import storage as google_storage
import os

def get_bytes(filename: str) -> bytes:
  gs_client = google_storage.Client.create_anonymous_client()
  bucket = gs_client.bucket("amsterdam-f2f-25")
  blob = bucket.get_blob(filename)
  file_content = blob.download_as_bytes()
  print(len(file_content))
  return file_content

def download_blob_to_file(blob_name: str, file_name: str):
    """Downloads a blob from the bucket to a local file."""
    os.makedirs('./assets', exist_ok=True)

    destination_file_name = './assets/' + file_name

    client = google_storage.Client.create_anonymous_client()
    bucket = client.bucket("amsterdam-f2f-25")
    blob = bucket.blob(blob_name)

    blob.download_to_filename(destination_file_name)
    print(f"Blob {blob_name} downloaded to {destination_file_name}.")


# Lab 1: Using Core APIs

This lab focuses on fundamental OSDU services like Entitlements, Legal, Schema, Search, Storage, and Dataset. We will perform operations like creating/managing groups, applying legal tags, exploring schemas, searching for records, and uploading/downloading data.

## OSDU Entitlements Service Lab

This lab demonstrates how to interact with the OSDU Entitlements Service. The Entitlements Service manages user groups and their access permissions within a data partition.

**Key Concepts:**
*   **Groups:** Collections of users identified by an email address (e.g., `my-group@data-partition.domain.com`).
*   **Members:** Users within a group, identified by their OSDU user ID.
*   **Roles:** Permissions within a group (e.g., `OWNER`, `MEMBER`).

**Documentation:**
*   [Entitlements Service](https://osdu.pages.opengroup.org/platform/security-and-compliance/entitlements/)
*   [Entitlements API](https://osdu.pages.opengroup.org/platform/security-and-compliance/entitlements/api/)
*   [API Specification](https://community.opengroup.org/osdu/platform/security-and-compliance/entitlements/-/raw/master/docs/api/entitlements_openapi.yaml?ref_type=heads)

### Setup: Entitlements Lab

This cell sets up the necessary variables for the Entitlements lab exercises.

**Dependencies:**
*   Requires the `requests` and `json` libraries (imported in the initial setup).
*   Relies on variables defined during the initial authentication: `osdu_endpoint`, `headers`, `user_id`, `entitlements_domain`, `osdu_data_partition_id`.

In [5]:
# Define the base endpoint for the Entitlements API
entitlements_endpoint = f"{osdu_endpoint}/api/entitlements/v2"
print(f"Using Entitlements API Endpoint: {entitlements_endpoint}")

# IMPORTANT: Define the email address of the user who is authenticated.
# This email MUST correspond to a valid user in the OSDU instance for adding/removing members.
# The {user_id} variable (a random number) is used to create unique group names during the lab.
# Example format: user.name@domain.com
# Replace this with the ACTUAL user email if needed for specific OSDU environments.
user_id_for_membership = f"user{user_id}" # Constructing a plausible email based on domain
print(f"Using user email for membership actions: {user_id_for_membership}")

# This variable will store the email of the group created in Step 2
group_email = None

Using Entitlements API Endpoint: https://osdu.osdu-bootcamp.com/api/entitlements/v2
Using user email for membership actions: user7073


### 1. List All Groups

Retrieve a list of all entitlement groups within the data partition.

**API Endpoint:** `GET /groups`
**Access needed:** `service.entitlements.user`

In [6]:
print("Attempting to list all groups...")
try:
    # Send a GET request to the /groups endpoint
    list_groups_response = requests.get(
        f"{entitlements_endpoint}/groups?all",
        headers=headers
    )
    # Raise an exception if the API returned an error (e.g., 4xx or 5xx)
    list_groups_response.raise_for_status()

    # Pretty print the JSON response
    print(f"List Groups Status Code: {list_groups_response.status_code}")
    print(json.dumps(list_groups_response.json(), indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error listing groups: {e}")
    # Print response body if available, helpful for debugging
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

Attempting to list all groups...
Error listing groups: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups?all (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2536dbf50>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 2. Create a Group

Create a new entitlements group. Group names must follow the pattern `<name>@<domain>.com`. The domain typically matches your data partition ID or organization. We incorporate the `user_id` number into the group name to avoid collisions between lab users.

**API Endpoint:** `POST /groups`
**Access needed:** `service.entitlements.admin`

| ⚠️ A **409 Client Error (Conflict)** response usually means that the group already exists. This is okay for the lab. |
|-------------------------------------------------------------------------------------------------------------------|

In [7]:
# Define the name and description for the new group
# Using user_id ensures the name is unique for each lab participant
group_name_prefix = f"lab-{user_id}-group"
group_description = f"Temporary group for lab exercise User {user_id}"
group_email = f"{group_name_prefix}@{entitlements_domain}"

# Prepare the request payload (body)
create_payload = {
    "name": group_name_prefix,
    "description": group_description
}

print(f"Attempting to create group with email: {group_email}")
print(f"Request Body: {json.dumps(create_payload)}")

try:
    # Send a POST request to create the group
    create_response = requests.post(
        f"{entitlements_endpoint}/groups",
        headers=headers,
        data=json.dumps(create_payload) # Send payload as JSON string
    )

    # Check for common non-fatal error (Conflict = already exists)
    if create_response.status_code == 409:
        print(f"Group '{group_email}' already exists (Status Code: 409). Proceeding...")
        # Optionally retrieve the existing group details if needed
        print(f"Response Body: {create_response.text}")
    else:
        # Raise exception for other errors
        create_response.raise_for_status()
        print(f"Group '{group_email}' created successfully (Status Code: {create_response.status_code}).")
        # Print the response from the server (usually confirms the group details)
        print(json.dumps(create_response.json(), indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error creating group {group_email}: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")
finally:
    # Ensure group_email is set even if creation failed with 409
    if group_email is None:
       group_email = f"{group_name_prefix}@{entitlements_domain}"
       print(f"Setting group_email to {group_email} for subsequent steps.")

Attempting to create group with email: lab-7073-group@None
Request Body: {"name": "lab-7073-group", "description": "Temporary group for lab exercise User 7073"}
Error creating group lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2518e2a20>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 3. Get Group Members

Retrieve the list of members for the group created (or identified) in the previous step. Since it was just created, it should be empty.

**API Endpoint:** `GET /groups/{group_email}/members`
**Access needed:** `service.entitlements.user`

In [8]:
# Ensure group_email is defined from the previous step
if group_email:
    print(f"Attempting to get members for group: {group_email}")
    try:
        # Send a GET request to the specific group's members endpoint
        get_members_response = requests.get(
            f"{entitlements_endpoint}/groups/{group_email}/members",
            headers=headers
        )
        get_members_response.raise_for_status()

        print(f"Get Members Status Code: {get_members_response.status_code}")
        print(f"Members for group '{group_email}':")
        print(json.dumps(get_members_response.json(), indent=2))

    except requests.exceptions.RequestException as e:
        print(f"Error getting members for group {group_email}: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Variable 'group_email' is not defined. Please run the 'Create a Group' step (Step 2) successfully first.")

Attempting to get members for group: lab-7073-group@None
Error getting members for group lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/lab-7073-group@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2518e24e0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 4. Add Member to Group

Add the current user (using their email `user_id_for_membership` defined in Setup) to the group with a specific role (`OWNER` or `MEMBER`).

**API Endpoint:** `POST /groups/{group_email}/members`
**Access needed:** `service.entitlements.admin` AND the caller must be an `OWNER` of the target group.

In [9]:
# Ensure group_email is defined
if group_email:
    # Prepare the payload to add a member
    member_payload = {
        "email": user_id_for_membership,
        "role": "MEMBER" # Role can be MEMBER or OWNER
    }

    print(f"Attempting to add member '{member_payload['email']}' with role '{member_payload['role']}' to group '{group_email}'...")
    print(f"Request Body: {json.dumps(member_payload)}")

    try:
        # Send a POST request to add the member
        add_member_response = requests.post(
            f"{entitlements_endpoint}/groups/{group_email}/members",
            headers=headers,
            data=json.dumps(member_payload)
        )
        add_member_response.raise_for_status()

        # A successful add operation usually returns 200 OK with an empty body or member details
        print(f"Add Member Status Code: {add_member_response.status_code}")
        print(f"Member '{member_payload['email']}' added to group '{group_email}' with role '{member_payload['role']}'.")
        # Response body might be empty or contain member details
        if add_member_response.text:
            print(f"Response Body: {json.dumps(add_member_response.json(), indent=2)}")
        else:
             print("Response Body: (empty)")

    except requests.exceptions.RequestException as e:
        print(f"Error adding member {member_payload['email']} to group {group_email}: {e}")
        # Note: A 403 Forbidden error might indicate the authenticated user is not an OWNER of the group.
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Variable 'group_email' is not defined. Please run the 'Create a Group' step (Step 2) successfully first.")

Attempting to add member 'user7073' with role 'MEMBER' to group 'lab-7073-group@None'...
Request Body: {"email": "user7073", "role": "MEMBER"}
Error adding member user7073 to group lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/lab-7073-group@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a2540>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 5. List Group Members (Again)

Retrieve the list of members for the group again. This time, it should show the member added in the previous step.

**API Endpoint:** `GET /groups/{group_email}/members`
**Access needed:** `service.entitlements.user`

In [10]:
# Ensure group_email is defined from the previous step
if group_email:
    print(f"Attempting to get members for group: {group_email} (after adding member)")
    try:
        # Send a GET request to the specific group's members endpoint
        get_members_response = requests.get(
            f"{entitlements_endpoint}/groups/{group_email}/members",
            headers=headers
        )
        get_members_response.raise_for_status()

        print(f"Get Members Status Code: {get_members_response.status_code}")
        print(f"Members for group '{group_email}':")
        print(json.dumps(get_members_response.json(), indent=2))

    except requests.exceptions.RequestException as e:
        print(f"Error getting members for group {group_email}: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Variable 'group_email' is not defined. Please run the 'Create a Group' step (Step 2) successfully first.")

Attempting to get members for group: lab-7073-group@None (after adding member)
Error getting members for group lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/lab-7073-group@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a3f50>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 6. Remove Member from Group

Remove the member (`user_id_for_membership`) that was added in Step 4.

**API Endpoint:** `DELETE /groups/{group_email}/members/{member_email}`
**Access needed:** `service.entitlements.admin` AND the caller must be an `OWNER` of the target group.

In [11]:
# Ensure group_email is defined
if group_email:
    member_to_remove = user_id_for_membership
    print(f"Attempting to remove member '{member_to_remove}' from group '{group_email}'...")

    try:
        # Send a DELETE request to remove the member
        remove_response = requests.delete(
            f"{entitlements_endpoint}/groups/{group_email}/members/{member_to_remove}",
            headers=headers
        )
        remove_response.raise_for_status()

        # A successful deletion usually returns 204 No Content
        print(f"Remove Member Status Code: {remove_response.status_code}")
        print(f"Member '{member_to_remove}' removed from group '{group_email}'.")
        # Response body is typically empty on success
        if remove_response.text:
            print(f"Response Body: {remove_response.text}")

    except requests.exceptions.RequestException as e:
        print(f"Error removing member {member_to_remove} from group {group_email}: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")

    # Verify removal by listing members again (Optional but recommended)
    print(f"\nVerifying removal by listing members for group: {group_email}")
    try:
        verify_response = requests.get(
            f"{entitlements_endpoint}/groups/{group_email}/members",
            headers=headers
        )
        verify_response.raise_for_status()
        print(f"Verification - Get Members Status Code: {verify_response.status_code}")
        print(f"Current members for group '{group_email}':")
        print(json.dumps(verify_response.json(), indent=2))
    except requests.exceptions.RequestException as e:
        print(f"Error verifying group members for {group_email}: {e}")
        if e.response is not None:
             print(f"Verification Response Body: {e.response.text}")

else:
    print("Variable 'group_email' is not defined. Please run the 'Create a Group' step (Step 2) successfully first.")

Attempting to remove member 'user7073' from group 'lab-7073-group@None'...
Error removing member user7073 from group lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/lab-7073-group@None/members/user7073 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a3770>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))

Verifying removal by listing members for group: lab-7073-group@None
Error verifying group members for lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/lab-7073-group@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2518e2c30>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 7. Delete Group

Delete the group created earlier in the lab.

**API Endpoint:** `DELETE /groups/{group_email}`
**Access needed:** `service.entitlements.admin` AND the caller must be an `OWNER` of the group.

In [12]:
# Ensure group_email is defined
if group_email:
    print(f"Attempting to delete group: {group_email}")
    try:
        # Send a DELETE request to remove the group
        # NOTE: The API spec shows DELETE /groups/{group_email} requires no specific headers beyond auth and data-partition-id
        delete_response = requests.delete(
            f"{entitlements_endpoint}/groups/{group_email}",
            headers=headers # Standard headers are sufficient
        )
        delete_response.raise_for_status()

        # Successful deletion usually returns 204 No Content
        print(f"Delete Group Status Code: {delete_response.status_code}")
        print(f"Group '{group_email}' deleted successfully.")
        # Response is typically empty on success
        if delete_response.text:
             print(f"Response Body: {delete_response.text}")

        # Clean up variable to prevent accidental reuse in subsequent runs
        print(f"Deleting variable group_email ('{group_email}') from notebook memory.")
        del group_email

    except requests.exceptions.RequestException as e:
        print(f"Error deleting group {group_email}: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
    except NameError:
        # This case handles if 'del group_email' was successful but cell is run again
        print("Variable 'group_email' no longer exists (likely already deleted). ")
else:
    # This case handles if group_email was never defined (e.g., Step 2 failed)
    print("Variable 'group_email' is not defined. Cannot delete group. Please run 'Create a Group' (Step 2) first or the group may have already been deleted.")


Attempting to delete group: lab-7073-group@None
Error deleting group lab-7073-group@None: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/lab-7073-group@None (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25170ccb0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### Challenge

1.  Create a new group named `challenge-{user_id}-users` (where `{user_id}` is your unique number) with the description "Challenge Group for {user_id}".
2.  Add two members to this group:
    *   Your own user email (`user_id_for_membership`) with the role `OWNER`.
    *   A *different* email address (e.g., `user{user_id}-teammate@{entitlements_domain}`) with the role `MEMBER`.
3.  List all members of the challenge group to verify additions.
4.  Remove the second member (`user{user_id}-teammate@{entitlements_domain}`) from the group.
5.  List the members again to verify the removal.
6.  Delete the challenge group.

In [13]:
# Code block for setting up challenge variables
print("--- Starting Entitlements Challenge ---")

# Define unique names and emails for the challenge
challenge_group_prefix = f"challenge-{user_id}-users"
challenge_group_desc = f"Challenge Group for {user_id}"
challenge_group_email = f"{challenge_group_prefix}@{entitlements_domain}"

# Member 1: Your own email (defined in Setup)
member1_email_challenge = user_id_for_membership
# Member 2: A different email address (doesn't have to be a real user for this exercise unless testing actual permissions)
member2_email_challenge = f"user{user_id}-teammate@{entitlements_domain}"

print(f"Challenge Group Email: {challenge_group_email}")
print(f"Challenge Member 1 (OWNER): {member1_email_challenge}")
print(f"Challenge Member 2 (MEMBER): {member2_email_challenge}")

# Use the standard 'headers' defined in the initial setup
# headers = { ... }

--- Starting Entitlements Challenge ---
Challenge Group Email: challenge-7073-users@None
Challenge Member 1 (OWNER): user7073
Challenge Member 2 (MEMBER): user7073-teammate@None


#### 💡 Solution Code Cells

##### 1. Create Challenge Group

In [14]:
print(f"\nChallenge Step 1: Creating group: {challenge_group_email}")
challenge_group_payload = {
    "name": challenge_group_prefix,
    "description": challenge_group_desc,
}

try:
    response = requests.post(
        f"{entitlements_endpoint}/groups",
        headers=headers,
        data=json.dumps(challenge_group_payload)
    )
    if response.status_code == 409:
        print(f"   Group '{challenge_group_email}' already exists (409). Proceeding...")
    else:
        response.raise_for_status()
        print(f"   Group '{challenge_group_email}' created successfully ({response.status_code}).")
        # print(json.dumps(response.json(), indent=2))
except requests.exceptions.RequestException as e:
    print(f"   Error creating challenge group: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 1: Creating group: challenge-7073-users@None
   Error creating challenge group: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a2240>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### Create Member 2 Group

- Challenge Member 1 (OWNER): user123
- Challenge Member 2 (MEMBER): user123-teammate@osdu.group

user123-teammate@osdu.group is recognized as a group (because it has 'osdu' and 'group'). If we have any other postfix (@gmail.com. @osdu1.group, etc), the following code is not mandatory.

In [15]:
challenge_group_email
member2_email_challenge
print(f"\nChallenge Step 1: Creating group: {member2_email_challenge}")
challenge_group_payload = {
    "name": member2_email_challenge.replace(f"@{osdu_data_partition_id}.{osdu_group_domain}", ""),
    "description": f"Description for {member2_email_challenge}",
}

try:
    response = requests.post(
        f"{entitlements_endpoint}/groups",
        headers=headers,
        data=json.dumps(challenge_group_payload)
    )
    if response.status_code == 409:
        print(f"   Group '{member2_email_challenge}' already exists (409). Proceeding...")
    else:
        response.raise_for_status()
        print(f"   Group '{member2_email_challenge}' created successfully ({response.status_code}).")
        # print(json.dumps(response.json(), indent=2))
except requests.exceptions.RequestException as e:
    print(f"   Error creating challenge group: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 1: Creating group: user7073-teammate@None
   Error creating challenge group: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a34a0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 2. Add Members to Challenge Group

In [16]:
# Note: Removed unused import 'from osdu_api.model.entitlements.group_member import GroupMember'
print("\nChallenge Step 2: Adding members...")

# Prepare payloads for each member
member1_payload_challenge = {
    "email": member1_email_challenge,
    "role": "OWNER"
}

member2_payload_challenge = {
    "email": member2_email_challenge,
    "role": "MEMBER"
}

# Add Member 1
try:
    print(f"   Adding Member 1: {member1_email_challenge} as {member1_payload_challenge['role']}")
    response1 = requests.post(
        f"{entitlements_endpoint}/groups/{challenge_group_email}/members",
        headers=headers,
        data=json.dumps(member1_payload_challenge)
    )
    response1.raise_for_status()
    print(f"      Success ({response1.status_code}).")
except requests.exceptions.RequestException as e:
    # Handle case where member might already exist in the group (common if re-running)
    # The API might return 400 or other codes depending on implementation if member exists.
    print(f"      Error adding member 1: {e}")
    if e.response is not None:
        print(f"      Response Body: {e.response.text}")

# Add Member 2
try:
    print(f"   Adding Member 2: {member2_email_challenge} as {member2_payload_challenge['role']}")
    response2 = requests.post(
        f"{entitlements_endpoint}/groups/{challenge_group_email}/members",
        headers=headers,
        data=json.dumps(member2_payload_challenge)
    )
    response2.raise_for_status()
    print(f"      Success ({response2.status_code}).")
except requests.exceptions.RequestException as e:
    print(f"      Error adding member 2: {e}")
    if e.response is not None:
        print(f"      Response Body: {e.response.text}")


Challenge Step 2: Adding members...
   Adding Member 1: user7073 as OWNER
      Error adding member 1: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/challenge-7073-users@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25170e570>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))
   Adding Member 2: user7073-teammate@None as MEMBER
      Error adding member 2: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/challenge-7073-users@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25170c200>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 3. List Challenge Group Members

In [17]:
print(f"\nChallenge Step 3: Listing members of {challenge_group_email}")
try:
    list_members_response = requests.get(
        f"{entitlements_endpoint}/groups/{challenge_group_email}/members",
        headers=headers
    )
    list_members_response.raise_for_status()
    print(f"   List Members Status Code: {list_members_response.status_code}")
    print("   Current Members:")
    print(json.dumps(list_members_response.json(), indent=2))
except requests.exceptions.RequestException as e:
     print(f"   Error listing challenge group members: {e}")
     if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 3: Listing members of challenge-7073-users@None
   Error listing challenge group members: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/challenge-7073-users@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25170dd30>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 4. Remove Member 2 from Challenge Group

In [18]:
print(f"\nChallenge Step 4: Removing member {member2_email_challenge} from {challenge_group_email}")
try:
    remove_response = requests.delete(
        f"{entitlements_endpoint}/groups/{challenge_group_email}/members/{member2_email_challenge}",
        headers=headers
    )
    remove_response.raise_for_status()
    print(f"   Member {member2_email_challenge} removed successfully ({remove_response.status_code}).")
except requests.exceptions.RequestException as e:
    print(f"   Error removing member 2: {e}")
    if e.response is not None:
        # A 404 might mean the member was already removed or never added successfully
        print(f"   Response Body: {e.response.text}")


Challenge Step 4: Removing member user7073-teammate@None from challenge-7073-users@None
   Error removing member 2: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/challenge-7073-users@None/members/user7073-teammate@None (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a3140>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 5. List Challenge Group Members Again

In [19]:
print(f"\nChallenge Step 5: Listing members of {challenge_group_email} (after removal)")
try:
    list_members_response_after = requests.get(
        f"{entitlements_endpoint}/groups/{challenge_group_email}/members",
        headers=headers
    )
    list_members_response_after.raise_for_status()
    print(f"   List Members Status Code: {list_members_response_after.status_code}")
    print("   Current Members:")
    print(json.dumps(list_members_response_after.json(), indent=2))
except requests.exceptions.RequestException as e:
     print(f"   Error listing challenge group members: {e}")
     if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 5: Listing members of challenge-7073-users@None (after removal)
   Error listing challenge group members: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/challenge-7073-users@None/members (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25170dfd0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 6. Delete Challenge Group

In [20]:
print(f"\nChallenge Step 6: Deleting group {challenge_group_email}")
try:
    delete_response = requests.delete(
        f"{entitlements_endpoint}/groups/{challenge_group_email}",
        headers=headers
    )
    delete_response.raise_for_status()
    print(f"   Group '{challenge_group_email}' deleted successfully ({delete_response.status_code}).")
except requests.exceptions.RequestException as e:
    print(f"   Error deleting challenge group: {e}")
    if e.response is not None:
        # A 404 might mean it was already deleted
        print(f"   Response Body: {e.response.text}")

print("--- Entitlements Challenge Complete ---")


Challenge Step 6: Deleting group challenge-7073-users@None
   Error deleting challenge group: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/challenge-7073-users@None (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e251768ad0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))
--- Entitlements Challenge Complete ---


##### Delete Member 2 Group

In [21]:
print(f"\nChallenge Step 6: Deleting group {member2_email_challenge}")
try:
    delete_response = requests.delete(
        f"{entitlements_endpoint}/groups/{member2_email_challenge}",
        headers=headers
    )
    delete_response.raise_for_status()
    print(f"   Group '{challenge_group_email}' deleted successfully ({delete_response.status_code}).")
except requests.exceptions.RequestException as e:
    print(f"   Error deleting challenge group: {e}")
    if e.response is not None:
        # A 404 might mean it was already deleted
        print(f"   Response Body: {e.response.text}")

print("--- Entitlements Challenge Complete ---")


Challenge Step 6: Deleting group user7073-teammate@None
   Error deleting challenge group: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/entitlements/v2/groups/user7073-teammate@None (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25170e390>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))
--- Entitlements Challenge Complete ---


## OSDU Legal Service Lab

This lab exercise covers the usage of the OSDU Legal Service. You will learn how to create, retrieve, list, update, validate, and delete legal tags. Legal tags are crucial for associating data records with legal compliance information (e.g., data origin, export controls, usage restrictions).

**Key Concepts:**
*   **Legal Tag:** A metadata tag containing properties related to legal compliance.
*   **Properties:** Attributes within a legal tag defining its specific compliance details (e.g., `countryOfOrigin`, `expirationDate`, `personalData`).

**Documentation:**
*   [Legal Service Concepts](https://osdu.pages.opengroup.org/platform/security-and-compliance/legal/)
*   [Legal Service API Usage](https://osdu.pages.opengroup.org/platform/security-and-compliance/legal/api/)
*   [API Specification](https://community.opengroup.org/osdu/platform/security-and-compliance/legal/-/blob/master/openapi/legal_openapi.yaml)

##### 6. Delete Challenge Group

### Setup: Legal Lab

This cell sets up the necessary variables for the Legal Service lab exercises.

**Dependencies:**
*   Requires the `requests` and `json` libraries (imported in the initial setup).
*   Relies on variables defined during the initial authentication: `osdu_endpoint`, `headers`, `user_id`, `osdu_data_partition_id`.

In [22]:
# Define the base endpoint for the Legal API
legal_endpoint = f"{osdu_endpoint}/api/legal/v1"
print(f"Using Legal API Endpoint: {legal_endpoint}")

# Define a unique tag name using the user_id to avoid conflicts
user_tag_name = f'{osdu_data_partition_id}-lab-tag-{user_id}'
user_tag_description = f"Lab tag created by user {user_id}"
print(f"Using Legal Tag Name: {user_tag_name}")

# This variable will store the name of the tag created in Step 1
created_tag_name = None

Using Legal API Endpoint: https://osdu.osdu-bootcamp.com/api/legal/v1
Using Legal Tag Name: osdu-lab-tag-7073


### 1. Create Legal Tag

Create a new legal tag with specified properties.

**API Endpoint:** `POST /legaltags`
**Access needed:** `service.legal.editor`

| ⚠️ A **409 Client Error (Conflict)** response usually means that the tag already exists. This is okay for the lab. |
|-------------------------------------------------------------------------------------------------------------------|

In [23]:
# Define the payload for the new legal tag
create_payload = {
    "name": user_tag_name,
    "description": user_tag_description,
    "properties": {
        "contractId": f"CID-{user_id}-A1",
        "countryOfOrigin": ["US"], # Must be an ISO 3166-1 alpha-2 country code
        "dataType": "Public Domain Data", # Value must exist in reference data 'LegalTagPropertyType' for 'dataType'
        "expirationDate": "2099-12-31", # Format YYYY-MM-DD
        "exportClassification": "EAR99", # Value must exist in reference data 'LegalTagPropertyType' for 'exportClassification'
        "originator": f"LabUser-{user_id}",
        "personalData": "No Personal Data", # Value must exist in reference data 'LegalTagPropertyType' for 'personalData'
        "securityClassification": "Public" # Value must exist in reference data 'LegalTagPropertyType' for 'securityClassification'
    }
}

print(f"Attempting to create legal tag: {user_tag_name}")
print(f"Request Body: {json.dumps(create_payload, indent=2)}")

try:
    # Send POST request to create the legal tag
    create_response = requests.post(
        f"{legal_endpoint}/legaltags",
        headers=headers,
        data=json.dumps(create_payload) # Send payload as JSON string
    )

    # Handle potential conflict (already exists)
    if create_response.status_code == 409:
        print(f"Legal tag '{user_tag_name}' already exists (Status Code: 409). Proceeding...")
        # Attempt to retrieve the existing tag to confirm structure (optional)
        print(f"Response Body: {create_response.text}")
        created_tag_name = user_tag_name # Assume it exists with the correct name
    else:
        # Raise exception for other HTTP errors
        create_response.raise_for_status()
        print(f"Legal tag creation successful (Status Code: {create_response.status_code}).")
        created_tag_data = create_response.json()
        print("Response Body:")
        print(json.dumps(created_tag_data, indent=2))
        # Store the name from the response (might differ if case-sensitivity rules apply)
        created_tag_name = created_tag_data.get("name")

except requests.exceptions.RequestException as e:
    print(f"Error creating legal tag '{user_tag_name}': {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")
finally:
     # Ensure the variable is set if creation succeeded or conflict occurred
    if created_tag_name:
        print(f"Stored created_tag_name: {created_tag_name}")
    else:
        print(f"Failed to create or confirm existence of legal tag '{user_tag_name}'. Subsequent steps might fail.")

Attempting to create legal tag: osdu-lab-tag-7073
Request Body: {
  "name": "osdu-lab-tag-7073",
  "description": "Lab tag created by user 7073",
  "properties": {
    "contractId": "CID-7073-A1",
    "countryOfOrigin": [
      "US"
    ],
    "dataType": "Public Domain Data",
    "expirationDate": "2099-12-31",
    "exportClassification": "EAR99",
    "originator": "LabUser-7073",
    "personalData": "No Personal Data",
    "securityClassification": "Public"
  }
}
Error creating legal tag 'osdu-lab-tag-7073': HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e251768fe0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))
Failed to create or confirm existence of legal tag 'osdu-lab-tag-7073'. Subsequent steps might fail.


### 2. Get Legal Tag

Retrieve the details of the legal tag created (or confirmed existing) in the previous step by its name.

**API Endpoint:** `GET /legaltags/{tagName}`
**Access needed:** `service.legal.user`

In [24]:
# Ensure created_tag_name has a value from the previous step
if created_tag_name:
    print(f"Attempting to retrieve legal tag: {created_tag_name}")
    try:
        # Send GET request to retrieve the specific tag
        get_response = requests.get(
            f"{legal_endpoint}/legaltags/{created_tag_name}",
            headers=headers
        )
        get_response.raise_for_status() # Check for HTTP errors (e.g., 404 Not Found)

        print(f"Legal tag retrieval successful (Status Code: {get_response.status_code}).")
        print("Retrieved Tag Details:")
        print(json.dumps(get_response.json(), indent=2))

    except requests.exceptions.RequestException as e:
        print(f"Error retrieving legal tag '{created_tag_name}': {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Variable 'created_tag_name' is not defined. Cannot retrieve tag. Please ensure Step 1 ran successfully.")

Variable 'created_tag_name' is not defined. Cannot retrieve tag. Please ensure Step 1 ran successfully.


### 3. List Legal Tags

List all available legal tags in the data partition. This may return many tags depending on the environment.

**API Endpoint:** `GET /legaltags`
**Access needed:** `service.legal.user`

In [25]:
print("Attempting to list all legal tags...")
try:
    # Send GET request to list all tags
    list_response = requests.get(
        f"{legal_endpoint}/legaltags",
        headers=headers
    )
    list_response.raise_for_status()

    print(f"List legal tags successful (Status Code: {list_response.status_code}).")
    tag_list_data = list_response.json()

    # Display the result (can be large)
    if 'legalTags' in tag_list_data and isinstance(tag_list_data['legalTags'], list):
        print(f"Found {len(tag_list_data['legalTags'])} legal tags.")
        # Optionally print only the names or a subset if the list is too long
        # print("First 5 tags:")
        # print(json.dumps(tag_list_data['legalTags'][:5], indent=2))
        print("Full list response:")
        print(json.dumps(tag_list_data, indent=2))
    else:
        print("Unexpected response format:")
        print(json.dumps(tag_list_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error listing legal tags: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

Attempting to list all legal tags...
Error listing legal tags: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25176ad50>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 4. Update Legal Tag

Update properties of the legal tag created earlier. Note that `PUT /legaltags` typically replaces the entire tag definition, so we retrieve the existing tag first, modify it, and then send the complete updated object.

**API Endpoint:** `PUT /legaltags`
**Access needed:** `service.legal.editor`

In [26]:
print(f"Attempting to update legal tag: {created_tag_name}")

current_tag_data = None

# Modify the retrieved data
current_tag_data = {
    "name": user_tag_name,
    "description": "Updated legal tag created for SDK lab exercise.",
    "contractId": "A1234-updated",
    "expirationDate": "2199-12-31",
}

print(f"   Attempting to PUT updated legal tag: {created_tag_name}")
print(f"   Update Request Body: {json.dumps(current_tag_data, indent=2)}")

try:
    update_response = requests.put(
        f"{legal_endpoint}/legaltags", # Note: PUT is to the base /legaltags endpoint
        headers=headers,
        data=json.dumps(current_tag_data) # Send the entire modified object
    )
    update_response.raise_for_status()

    print(f"   Legal tag update successful (Status Code: {update_response.status_code}).")
    print("   Updated Tag Details:")
    print(json.dumps(update_response.json(), indent=2))

except requests.exceptions.RequestException as e:
    print(f"   Error updating legal tag '{created_tag_name}': {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")

Attempting to update legal tag: None
   Attempting to PUT updated legal tag: None
   Update Request Body: {
  "name": "osdu-lab-tag-7073",
  "description": "Updated legal tag created for SDK lab exercise.",
  "contractId": "A1234-updated",
  "expirationDate": "2199-12-31"
}
   Error updating legal tag 'None': HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25176b3e0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 5. Validate Legal Tags

Validate a list of tag names to check if they exist and are valid according to the Legal Service. The API returns a list containing only the *invalid* tag names from the input list. An empty `invalidLegalTags` list in the response means all provided tags were valid.

**API Endpoint:** `POST /legaltags:validate`
**Access needed:** `service.legal.user`

In [27]:
# Ensure created_tag_name has a value
if created_tag_name:
    # Prepare payload with tags to validate
    # Include the valid tag created earlier and a clearly invalid one
    invalid_test_tag_name = f"non-existent-tag-{user_id}"
    tags_to_validate_payload = {
        "names": [
            created_tag_name,
            invalid_test_tag_name
        ]
    }

    print(f"Attempting to validate tags: {tags_to_validate_payload['names']}")
    print(f"Request Body: {json.dumps(tags_to_validate_payload, indent=2)}")

    try:
        validate_response = requests.post(
            f"{legal_endpoint}/legaltags:validate",
            headers=headers,
            data=json.dumps(tags_to_validate_payload)
        )
        validate_response.raise_for_status()

        print(f"Validation request successful (Status Code: {validate_response.status_code}).")
        validation_result = validate_response.json()
        print("Validation Results:")
        print(json.dumps(validation_result, indent=2))

        # Interpretation of results
        if 'invalidLegalTags' in validation_result:
            if not validation_result['invalidLegalTags']:
                print(f"   Interpretation: All tags in the request ({tags_to_validate_payload['names']}) are valid.")
            else:
                print(f"   Interpretation: The following tags were reported as invalid: {validation_result['invalidLegalTags']}")
                # Check if the expected invalid tag is present
                if invalid_test_tag_name in validation_result['invalidLegalTags']:
                     print(f"      As expected, '{invalid_test_tag_name}' was found to be invalid.")
                # Check if the supposedly valid tag was reported as invalid (problem?)
                if created_tag_name in validation_result['invalidLegalTags']:
                     print(f"      WARNING: Tag '{created_tag_name}', which should be valid, was reported as invalid!")
        else:
             print("   Warning: 'invalidLegalTags' key not found in the response.")

    except requests.exceptions.RequestException as e:
        print(f"Error validating legal tags: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Variable 'created_tag_name' is not defined. Cannot validate tag. Please ensure Step 1 ran successfully.")

Variable 'created_tag_name' is not defined. Cannot validate tag. Please ensure Step 1 ran successfully.


### 6. Get Properties Schema

Retrieve the schema definition for the `properties` object within a legal tag. This defines which properties are allowed and their expected data types or allowed values (often linked to OSDU reference data).

**API Endpoint:** `GET /legaltags:properties`
**Access needed:** `service.legal.user`

In [28]:
print("Attempting to get legal tag properties schema...")
try:
    properties_response = requests.get(
        f"{legal_endpoint}/legaltags:properties",
        headers=headers
    )
    properties_response.raise_for_status()

    print(f"Get properties schema successful (Status Code: {properties_response.status_code}).")
    print("Legal Tag Properties Schema:")
    print(json.dumps(properties_response.json(), indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error getting legal tag properties schema: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

Attempting to get legal tag properties schema...
Error getting legal tag properties schema: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags:properties (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2518e3f50>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 7. Delete Legal Tag

Delete the legal tag created during this lab.

**API Endpoint:** `DELETE /legaltags/{tagName}`
**Access needed:** `service.legal.admin`

In [29]:
# Ensure created_tag_name has a value
if created_tag_name:
    print(f"Attempting to delete legal tag: {created_tag_name}")
    try:
        # Send DELETE request
        delete_response = requests.delete(
            f"{legal_endpoint}/legaltags/{created_tag_name}",
            headers=headers
        )
        # Successful deletion usually returns 204 No Content
        delete_response.raise_for_status()

        print(f"Legal tag deletion successful (Status Code: {delete_response.status_code}).")
        if delete_response.text:
            print(f"Response Body: {delete_response.text}") # Should ideally be empty

        # --- Verification ---
        print(f"\nVerifying deletion by trying to retrieve tag: {created_tag_name}...")
        try:
            verify_response = requests.get(
                f"{legal_endpoint}/legaltags/{created_tag_name}",
                headers=headers
            )
            # We EXPECT this to fail with a 404 Not Found error
            if verify_response.status_code == 404:
                print(f"   Verification successful: Received expected 404 Not Found for tag '{created_tag_name}'.")
            else:
                # If it succeeds or fails with a different error, the deletion might not have worked
                verify_response.raise_for_status() # Raise other errors
                print(f"   Verification Warning: Retrieval after delete returned {verify_response.status_code} instead of 404.")
                print(f"   Response Body: {verify_response.text}")

        except requests.exceptions.RequestException as ve:
            # Specifically catch the expected 404 during verification
            if ve.response is not None and ve.response.status_code == 404:
                 print(f"   Verification successful: GET request failed with expected 404 Not Found for tag '{created_tag_name}'.")
            else:
                 # Unexpected error during verification
                 print(f"   Verification Error: Unexpected error while trying to retrieve deleted tag: {ve}")
                 if ve.response is not None:
                      print(f"   Verification Response Body: {ve.response.text}")

    except requests.exceptions.RequestException as e:
        print(f"Error deleting legal tag '{created_tag_name}': {e}")
        if e.response is not None:
            # A 404 here means it was likely already deleted
            if e.response.status_code == 404:
                 print(f"   Tag '{created_tag_name}' seems to have been already deleted (received 404)." )
            else:
                 print(f"Response Body: {e.response.text}")

    # Clean up variable even if deletion failed (to prevent reuse issues)
    print(f"\nDeleting variable created_tag_name ('{created_tag_name}') from notebook memory.")
    del created_tag_name

else:
    print("Variable 'created_tag_name' is not defined. Cannot delete tag. Please ensure Step 1 ran successfully or check if it was already deleted.")

Variable 'created_tag_name' is not defined. Cannot delete tag. Please ensure Step 1 ran successfully or check if it was already deleted.


### Challenge

1.  Create a new legal tag named `{osdu_data_partition_id}-challenge-tag-{user_id}`.
    *   Set `countryOfOrigin` to `['CA']` (Canada).
    *   Set `expirationDate` to `2025-12-31`.
    *   Include other mandatory properties required by the schema (Hint: refer to the output of Step 6 or the initial create payload in Step 1, adapting values as needed).
2.  Retrieve this challenge tag and print its details.
3.  Update the tag's `properties`:
    *   Change `contractId` to `challenge-contract-999`.
    *   Change `expirationDate` to `2026-06-30`.
    *   Ensure other properties are preserved.
4.  Validate that your challenge tag name is valid.
5.  Delete the challenge tag and verify its deletion.

In [30]:
# Code block for setting up challenge variables
print("--- Starting Legal Service Challenge ---")

# Define unique name for the challenge tag
challenge_tag_name = f'{osdu_data_partition_id}-challenge-tag-{user_id}'
challenge_tag_description = f"Challenge Tag for user {user_id}"

print(f"Challenge Legal Tag Name: {challenge_tag_name}")

# Use the standard 'headers' and 'legal_endpoint' defined earlier

--- Starting Legal Service Challenge ---
Challenge Legal Tag Name: osdu-challenge-tag-7073


#### 💡 Solution Code Cells

###### 1. Create Challenge Tag

dataType and securityClassification [requirements](https://osdu.pages.opengroup.org/platform/security-and-compliance/legal/api/)

In [31]:
print(f"\nChallenge Step 1: Creating tag: {challenge_tag_name}")

# Define the payload, ensuring all required properties are included
challenge_create_payload = {
    "name": challenge_tag_name,
    "description": challenge_tag_description,
    "properties": {
        "contractId": f"CID-Challenge-{user_id}", # Example value
        "countryOfOrigin": ["CA"], # Challenge requirement
        "expirationDate": "2025-12-31", # Challenge requirement
        "dataType": "Third Party Data", # Example, check reference values if needed
        "originator": f"ChallengeUser-{user_id}", # Example value
        "personalData": "No Personal Data", # Example value
        "securityClassification": "Private", # Example value # UPDATED - Restricted
        "exportClassification": "EAR99" # Example value
    }
}
print(f"   Request Body: {json.dumps(challenge_create_payload, indent=2)}")

try:
    challenge_create_response = requests.post(
        f"{legal_endpoint}/legaltags",
        headers=headers,
        data=json.dumps(challenge_create_payload)
    )

    if challenge_create_response.status_code == 409:
        print(f"   Challenge tag '{challenge_tag_name}' already exists (409). Proceeding...")
    else:
        challenge_create_response.raise_for_status()
        print(f"   Challenge tag created successfully ({challenge_create_response.status_code}).")
        # print(f"   Response: {json.dumps(challenge_create_response.json(), indent=2)}")

except requests.exceptions.RequestException as e:
    print(f"   Error creating challenge tag: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 1: Creating tag: osdu-challenge-tag-7073
   Request Body: {
  "name": "osdu-challenge-tag-7073",
  "description": "Challenge Tag for user 7073",
  "properties": {
    "contractId": "CID-Challenge-7073",
    "countryOfOrigin": [
      "CA"
    ],
    "expirationDate": "2025-12-31",
    "dataType": "Third Party Data",
    "originator": "ChallengeUser-7073",
    "personalData": "No Personal Data",
    "securityClassification": "Private",
    "exportClassification": "EAR99"
  }
}
   Error creating challenge tag: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2517b4320>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 2. Retrieve Challenge Tag

In [32]:
print(f"\nChallenge Step 2: Retrieving tag: {challenge_tag_name}")
retrieved_challenge_tag_data = None # Initialize variable
try:
    challenge_get_response = requests.get(
        f"{legal_endpoint}/legaltags/{challenge_tag_name}",
        headers=headers
    )
    challenge_get_response.raise_for_status()
    retrieved_challenge_tag_data = challenge_get_response.json()
    print(f"   Challenge tag retrieved successfully ({challenge_get_response.status_code}).")
    print("   Retrieved Tag Details:")
    print(json.dumps(retrieved_challenge_tag_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"   Error retrieving challenge tag: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 2: Retrieving tag: osdu-challenge-tag-7073
   Error retrieving challenge tag: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags/osdu-challenge-tag-7073 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25176b830>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 3. Update Challenge Tag

update_payload['properties'] requirements.
- [link1](https://osdu.pages.opengroup.org/platform/security-and-compliance/legal/api/#updating-a-legaltag)
- [link2](https://community.opengroup.org/osdu/platform/security-and-compliance/legal/-/blob/master/docs/api/legal_openapi.yaml?ref_type=heads)

In [33]:
print(f"\nChallenge Step 3: Updating tag: {challenge_tag_name}")

# Check if we successfully retrieved the tag in the previous step
if retrieved_challenge_tag_data:
    # Modify the retrieved data
    update_payload = {}

    if 'properties' in retrieved_challenge_tag_data:
        update_payload['name'] = retrieved_challenge_tag_data['name']
        update_payload['contractId'] = "challenge-contract-999" # Update property
        update_payload['expirationDate'] = "2026-06-30" # Update property
        update_payload['description'] = f"{challenge_tag_description} - Updated" # Update description too
    # else:
        print("   Warning: 'properties' key not found in retrieved challenge tag data. Cannot update properties.")

    print(f"   Attempting to PUT updated challenge tag...")
    print(f"   Update Request Body: {json.dumps(update_payload, indent=2)}")
    try:
        challenge_update_response = requests.put(
            f"{legal_endpoint}/legaltags",
            headers=headers,
            data=json.dumps(update_payload)
        )
        challenge_update_response.raise_for_status()
        print(f"   Challenge tag update successful ({challenge_update_response.status_code}).")
        print("   Updated Tag Details:")
        print(json.dumps(challenge_update_response.json(), indent=2))
    except requests.exceptions.RequestException as e:
        print(f"   Error updating challenge tag: {e}")
        if e.response is not None:
            print(f"   Response Body: {e.response.text}")
else:
    print("   Skipping update because challenge tag data was not retrieved successfully in Step 2.")


Challenge Step 3: Updating tag: osdu-challenge-tag-7073
   Skipping update because challenge tag data was not retrieved successfully in Step 2.


##### 4. Validate Challenge Tag

In [34]:
print(f"\nChallenge Step 4: Validating tag: {challenge_tag_name}")

challenge_validate_payload = {"names": [challenge_tag_name]}
print(f"   Request Body: {json.dumps(challenge_validate_payload, indent=2)}")

try:
    challenge_validate_response = requests.post(
        f"{legal_endpoint}/legaltags:validate",
        headers=headers,
        data=json.dumps(challenge_validate_payload)
    )
    challenge_validate_response.raise_for_status()

    validation_result = challenge_validate_response.json()
    print(f"   Validation request successful ({challenge_validate_response.status_code}).")
    print("   Validation Results:")
    print(json.dumps(validation_result, indent=2))

    # Interpretation
    if 'invalidLegalTags' in validation_result and not validation_result['invalidLegalTags']:
        print(f"   Interpretation: Tag '{challenge_tag_name}' is valid.")
    else:
        print(f"   Interpretation: Tag '{challenge_tag_name}' reported as invalid (or unexpected response format). Invalid tags: {validation_result.get('invalidLegalTags')}")

except requests.exceptions.RequestException as e:
    print(f"   Error validating challenge tag: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")


Challenge Step 4: Validating tag: osdu-challenge-tag-7073
   Request Body: {
  "names": [
    "osdu-challenge-tag-7073"
  ]
}
   Error validating challenge tag: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags:validate (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2517b6690>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


##### 5. Delete Challenge Tag

In [35]:
print(f"\nChallenge Step 5: Deleting tag: {challenge_tag_name}")
try:
    challenge_delete_response = requests.delete(
        f"{legal_endpoint}/legaltags/{challenge_tag_name}",
        headers=headers
    )
    challenge_delete_response.raise_for_status()
    print(f"   Challenge tag deletion successful ({challenge_delete_response.status_code}).")

    # Verification
    print(f"   Verifying deletion by trying to retrieve tag: {challenge_tag_name}...")
    try:
        verify_response = requests.get(f"{legal_endpoint}/legaltags/{challenge_tag_name}", headers=headers)
        if verify_response.status_code == 404:
            print("      Verification successful: Received expected 404 Not Found.")
        else:
             verify_response.raise_for_status()
             print(f"      Verification Warning: Retrieval after delete returned {verify_response.status_code} instead of 404.")
             print(f"      Response Body: {verify_response.text}")
    except requests.exceptions.RequestException as ve:
        if ve.response is not None and ve.response.status_code == 404:
            print("      Verification successful: GET request failed with expected 404 Not Found.")
        else:
            print(f"      Verification Error: Unexpected error during verification GET: {ve}")

except requests.exceptions.RequestException as e:
    print(f"   Error deleting challenge tag: {e}")
    if e.response is not None:
        if e.response.status_code == 404:
            print(f"      Tag '{challenge_tag_name}' seems to have been already deleted (received 404 on delete attempt)." )
        else:
            print(f"   Response Body: {e.response.text}")

print("--- Legal Service Challenge Complete ---")


Challenge Step 5: Deleting tag: osdu-challenge-tag-7073
   Error deleting challenge tag: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/legal/v1/legaltags/osdu-challenge-tag-7073 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2519a39b0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))
--- Legal Service Challenge Complete ---


## OSDU Schema Service Lab

This lab explores the OSDU Schema Service, which is responsible for managing the definitions (schemas) of data types stored within the OSDU platform. Schemas define the structure, attributes, and data types for each kind of record (e.g., Wellbore, Log, Document).

**Key Concepts:**
*   **Schema:** A JSON document defining the structure of a specific data kind.
*   **Kind:** A unique identifier for a schema, typically following the pattern `{authority}:{source}:{type}:{version}` (e.g., `osdu:wks:master-data--Wellbore:1.0.0`).
*   **SchemaInfo:** Metadata about the schema itself (e.g., status, scope, author).

**Objectives:**
*   List available schemas.
*   Retrieve a specific schema definition.
*   *Attempt* to create a new custom schema (experimental/requires specific permissions).
*   *Attempt* to verify the creation of the custom schema.

**Documentation:**
*   [Schema Service Concepts](https://osdu.pages.opengroup.org/platform/system/schema-service/)
*   [API Specification](https://community.opengroup.org/osdu/platform/system/schema-service/-/blob/master/openapi/schema-service.yaml)

### Setup: Schema Lab

This cell sets up the necessary variables for the Schema Service lab exercises.

**Dependencies:**
*   Requires the `requests` and `json` libraries (imported in the initial setup).
*   Relies on variables defined during the initial authentication: `osdu_endpoint`, `headers`, `user_id`, `osdu_data_partition_id`.

In [36]:
# Define the base endpoint for the Schema Service API
schema_endpoint = f"{osdu_endpoint}/api/schema-service/v1"
print(f"Using Schema API Endpoint: {schema_endpoint}")

# Use the globally defined headers including Authorization and data-partition-id
# print(f"Using Headers: {json.dumps(headers, indent=2)}")

print(f"Reminder - Using unique identifier: {user_id}")

Using Schema API Endpoint: https://osdu.osdu-bootcamp.com/api/schema-service/v1
Reminder - Using unique identifier: 7073


### 1. List All Schemas

Retrieve a list of schemas registered in the connected OSDU instance for the current data partition.

**API Endpoint:** `GET /schema`
**Access needed:** `service.schema.viewer`

In [37]:
print("Attempting to list all schemas...")
try:
    # Send GET request to the /schemas endpoint
    list_schemas_response = requests.get(
        f"{schema_endpoint}/schema?limit=1500",
        headers=headers
        # No request body needed for GET
    )
    list_schemas_response.raise_for_status() # Check for HTTP errors

    print(f"List Schemas successful (Status Code: {list_schemas_response.status_code}).")
    schemas_data = list_schemas_response.json()

    # The response typically contains 'schemas' (list of kinds), 'offset', 'count', 'totalCount'
    total_count = schemas_data.get("totalCount", "N/A")
    returned_count = schemas_data.get("count", "N/A")
    print(f"Total schemas reported: {total_count}")
    print(f"Schemas returned in this response: {returned_count}")

    # Print the full response or just the list of schema kinds
    print("Schema Kinds Returned:")
    if 'schemaInfos' in schemas_data and isinstance(schemas_data['schemaInfos'], list):
       # Print first 10 kinds for brevity
       print(json.dumps(schemas_data['schemaInfos'][:10], indent=2))
       if len(schemas_data['schemaInfos']) > 10:
           print("... (list truncated)")
    else:
        print("Could not find 'schemaInfos' list in response:")
        print(json.dumps(schemas_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error listing schemas: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

Attempting to list all schemas...
Error listing schemas: HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/schema-service/v1/schema?limit=1500 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e2517b62a0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 2. Get a Specific Schema

Retrieve the full definition (JSON structure) of a specific schema using its `kind`.

**API Endpoint:** `GET /schemas/{kind}`
**Access needed:** `service.schema.viewer`

In [38]:
# Specify the kind of the schema to retrieve
schema_kind_to_get = "osdu:wks:master-data--Wellbore:1.0.0" # Example: Wellbore schema
# Alternative example: "osdu:wks:work-product-component--WellLog:1.0.0"

print(f"Attempting to retrieve schema definition for kind: {schema_kind_to_get}...")
try:
    # Send GET request to the specific schema kind endpoint
    get_schema_response = requests.get(
        f"{schema_endpoint}/schema/{schema_kind_to_get}",
        headers=headers
    )
    get_schema_response.raise_for_status() # Check for errors (e.g., 404 if kind not found)

    print(f"Successfully retrieved schema (Status Code: {get_schema_response.status_code}).")
    print(f"Definition for schema '{schema_kind_to_get}':")
    print(json.dumps(get_schema_response.json(), indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error retrieving schema '{schema_kind_to_get}': {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

Attempting to retrieve schema definition for kind: osdu:wks:master-data--Wellbore:1.0.0...
Error retrieving schema 'osdu:wks:master-data--Wellbore:1.0.0': HTTPSConnectionPool(host='osdu.osdu-bootcamp.com', port=443): Max retries exceeded with url: /api/schema-service/v1/schema/osdu:wks:master-data--Wellbore:1.0.0 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x79e25176a4b0>, 'Connection to osdu.osdu-bootcamp.com timed out. (connect timeout=None)'))


### 3. Create a Custom Schema

Define and attempt to register a new custom schema. This is typically an administrative task and requires specific permissions (`service.schema.editor`). The exact required fields in the `schema` block can vary.

**This step is likely to fail without proper setup and permissions.**

**API Endpoint:** `POST /schemas`
**Access needed:** `service.schema.editor`

In [39]:
# Define a unique kind for the custom schema using the data partition and user_id
custom_schema_authority = osdu_data_partition_id # Using data partition as authority
custom_schema_source = "lab-custom"
custom_schema_type = f"my-custom-type-{user_id}"
custom_schema_version = "1.0.0"
custom_schema_kind = f"{custom_schema_authority}:{custom_schema_source}:{custom_schema_type}:{custom_schema_version}"

# Define the schema payload (structure based on OSDU standards)
custom_schema_payload = {
    "schemaInfo": {
        "schemaIdentity": {
            "authority": custom_schema_authority,
            "source": custom_schema_source,
            "entityType": custom_schema_type,
            "schemaVersionMajor": 1,
            "schemaVersionMinor": 0,
            "schemaVersionPatch": 0,
            "id": f"uri:{custom_schema_kind}" # Often derived from the kind
        },
        "status": "DEVELOPMENT", # Common statuses: DEVELOPMENT, STABLE, OBSOLETE
        "scope": "INTERNAL", # Or SHARED
        "supersededBy": None
        # "createdBy" and "dateCreated" are usually set by the service
    },
    "schema": { # The actual schema definition (JSON Schema format)
        "$schema": "http://json-schema.org/draft-07/schema#",
        "title": f"My Custom Type {user_id}",
        "description": "A custom schema created for the OSDU lab.",
        "type": "object",
        "properties": {
            "CustomID": {
                "type": "string",
                "description": "A unique identifier for this custom record."
            },
            "CustomName": {
                "type": "string",
                "description": "A name for this custom record."
            },
            "MeasurementValue": {
                "type": "number",
                "description": "A measurement value."
            }
            # Add other properties as needed
        },
        "required": ["CustomID"] # Specify mandatory fields
    }
}

print(f"Attempting to create custom schema with kind: {custom_schema_kind}...")
print(f"Request Body: {json.dumps(custom_schema_payload, indent=2)}")
print("\n--- NOTE: This operation requires specific permissions and may fail. ---")

try:
    # Send POST request to create the schema
    create_custom_schema_response = requests.post(
        f"{schema_endpoint}/schema",
        headers=headers,
        data=json.dumps(custom_schema_payload)
    )
    create_custom_schema_response.raise_for_status()

    print(f"Custom schema creation request successful (Status Code: {create_custom_schema_response.status_code}).")
    print("Schema creation response:")
    # Response might be empty or confirm the kind/details
    if create_custom_schema_response.text:
        print(json.dumps(create_custom_schema_response.json(), indent=2))
    else:
        print("(Empty Response Body)")
    print("Note: Schema creation might take time to become fully available.")

except requests.exceptions.RequestException as e:
    print(f"Error creating custom schema '{custom_schema_kind}': {e}")
    if e.response is not None:
        print(f"Response Status Code: {e.response.status_code}")
        print(f"Response Body: {e.response.text}")
        if e.response.status_code == 403:
             print("   Hint: Status code 403 (Forbidden) usually indicates missing 'service.schema.editor' permissions.")
        elif e.response.status_code == 400:
             print("   Hint: Status code 400 (Bad Request) often indicates an invalid payload/schema structure.")

Attempting to create custom schema with kind: osdu:lab-custom:my-custom-type-7073:1.0.0...
Request Body: {
  "schemaInfo": {
    "schemaIdentity": {
      "authority": "osdu",
      "source": "lab-custom",
      "entityType": "my-custom-type-7073",
      "schemaVersionMajor": 1,
      "schemaVersionMinor": 0,
      "schemaVersionPatch": 0,
      "id": "uri:osdu:lab-custom:my-custom-type-7073:1.0.0"
    },
    "status": "DEVELOPMENT",
    "scope": "INTERNAL",
    "supersededBy": null
  },
  "schema": {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "My Custom Type 7073",
    "description": "A custom schema created for the OSDU lab.",
    "type": "object",
    "properties": {
      "CustomID": {
        "type": "string",
        "description": "A unique identifier for this custom record."
      },
      "CustomName": {
        "type": "string",
        "description": "A name for this custom record."
      },
      "MeasurementValue": {
        "type": "number",
  

### 4. Verify Custom Schema Creation
Attempt to retrieve the custom schema created in the previous step to verify it exists. This depends entirely on the success of Step 3.

**API Endpoint:** `GET /schemas/{kind}`
**Access needed:** `service.schema.viewer`

In [ ]:
# Use the 'custom_schema_kind' defined in the previous step
print(f"Attempting to verify custom schema creation by retrieving kind: {custom_schema_kind}...")
print("--- NOTE: This will only succeed if Step 3 was successful. ---")

try:
    # Send GET request to retrieve the custom schema
    verify_schema_response = requests.get(
        f"{schema_endpoint}/schema/{custom_schema_kind}",
        headers=headers
    )
    verify_schema_response.raise_for_status() # Expect 404 if Step 3 failed

    print(f"Successfully retrieved custom schema (Status Code: {verify_schema_response.status_code}). Verification PASSED.")
    print(f"Definition for schema '{custom_schema_kind}':")
    print(json.dumps(verify_schema_response.json(), indent=2))

except requests.exceptions.RequestException as e:
    print(f"Error retrieving custom schema '{custom_schema_kind}' (Verification FAILED): {e}")
    if e.response is not None:
        print(f"Response Status Code: {e.response.status_code}")
        print(f"Response Body: {e.response.text}")
        if e.response.status_code == 404:
            print("   Hint: Status code 404 (Not Found) is expected if the schema was not created successfully in Step 3.")
    else:
        # Handle cases where there's no response object (e.g., network error)
        print("   No response received from the server.")

Attempting to verify custom schema creation by retrieving kind: osdu:lab-custom:my-custom-type-7073:1.0.0...
--- NOTE: This will only succeed if Step 3 was successful. ---


### Challenge

Retrieve all registered schema *versions* for the `master-data--Wellbore` entity type, which belongs to the `osdu:wks` authority and source. Print their kinds.

**Hint:** You will need to use the results from Step 1 (List All Schemas) and filter the kinds based on the authority, source, and entity type pattern.

In [ ]:
# Challenge: Find all versions of osdu:wks:master-data--Wellbore
target_authority = "osdu"
target_source = "wks"
target_entity = "master-data--Wellbore"
target_prefix = f"{target_authority}:{target_source}:{target_entity}:"

print(f"Challenge: Finding all schema kinds starting with prefix: '{target_prefix}'...")
print("--- NOTE: This requires Step 1 (List All Schemas) to have run successfully. ---")

# Placeholder for the results
wellbore_schema_versions = []

# --- Add your implementation below ---
# 1. Reuse the 'list_schemas_response' from Step 1 if it was successful
# 2. If not, optionally re-run the GET /schemas call here.
# 3. Extract the list of schema kinds from the response.
# 4. Iterate through the list and check if each kind starts with 'target_prefix'.
# 5. Add matching kinds to the 'wellbore_schema_versions' list.

# Example Implementation (assuming list_schemas_response from Step 1 is available):
try:
    # Check if the response object from Step 1 exists and was successful
    if 'list_schemas_response' in locals() and list_schemas_response.status_code == 200:
        print("   Using schema list data from Step 1.")
        schemas_data = list_schemas_response.json()
        if 'schemaInfos' in schemas_data and isinstance(schemas_data['schemaInfos'], list):
            all_schema_kinds = schemas_data['schemaInfos']
            for kind in all_schema_kinds:
                authority = kind['schemaIdentity']['authority']
                source = kind['schemaIdentity']['source']
                entityType = kind['schemaIdentity']['entityType']

                if (target_authority == authority) and (target_source == source) and (target_entity == entityType):
                    wellbore_schema_versions.append(kind)
        else:
            print("   Error: 'schemaInfos' list not found in the Step 1 response data.")
    else:
        print("   Warning: Schema list from Step 1 not available or was unsuccessful. Cannot perform filtering.")

except Exception as e:
    print(f"   An unexpected error occurred during filtering: {e}")

# --- End of implementation ---

# Print the results
print("\nChallenge Results:")
if wellbore_schema_versions:
    print("Found Wellbore Schema Versions:")
    # Sort for better readability
    for kind in wellbore_schema_versions:
        print(f"  - {kind}")
else:
    print("Could not find any schema versions matching the criteria, or Step 1 failed.")

print("--- Schema Service Challenge Complete ---")

## OSDU Search Service Lab

This lab demonstrates how to use the OSDU Search Service to find data records within the platform using various query mechanisms, including keyword search, text search, spatial filters, sorting, and pagination.

**Key Concepts:**
*   **Kind:** Specifies the type of record to search for (e.g., `osdu:wks:master-data--Wellbore:1.0.0`). Wildcards (`*`) can be used.
*   **Query:** The search criteria. Can be a simple wildcard (`*`) or use Apache Lucene syntax for more complex text searches (e.g., `data.FacilityName:"MyWell" AND data.Status:"Active"`).
*   **Limit:** The maximum number of results to return in a single request.
*   **Cursor:** A pointer used for pagination to retrieve large result sets page by page.
*   **Spatial Filter:** Allows searching for records based on geographic location (Point/Distance, Bounding Box, Polygon).
*   **Aggregation:** Groups results based on a specific field (e.g., count records by `kind`).

**Permissions Note:** Successful searching requires the `service.search.user` permission. However, the *results returned* also depend on the user having `VIEWER` permissions (defined in the record's Access Control List - ACL) for the individual records found by the search query.

**Documentation:**
*   [Search Service Usage](https://osdu.pages.opengroup.org/platform/system/search-service/)
*   [API Specification](https://community.opengroup.org/osdu/platform/system/search-service/-/blob/master/open-api/search_openapi.yaml)

### Setup: Search Lab

This cell sets up the necessary variables and helper functions for the Search Service lab exercises.

**Dependencies:**
*   Requires the `requests`, `json`, and `pandas` libraries.
*   Relies on variables defined during the initial authentication: `osdu_endpoint`, `headers`, `osdu_data_partition_id`.

In [ ]:
import pandas as pd # Ensure pandas is imported
import requests
import json

# Define the base endpoint for the Search API V2
search_endpoint = f"{osdu_endpoint}/api/search/v2"
print(f"Using Search API Endpoint: {search_endpoint}")

# Use the standard 'headers' defined in the initial setup
# print(f"Using Headers: {json.dumps(headers, indent=2)}")

# Helper function to format and print aggregation results
def print_agg_list_result(response: requests.Response):
  """Parses aggregation results from a search response and prints as a Markdown table."""
  try:
    response.raise_for_status() # Check for HTTP errors first
    data = response.json()
    # Check if 'aggregations' key exists and is a non-empty list
    if 'aggregations' in data and isinstance(data['aggregations'], list) and data['aggregations']:
      aggregation = data['aggregations']
      # Dynamically get keys from the first item assuming structure is consistent
      columns = list(aggregation[0].keys())
      df = pd.DataFrame(aggregation, columns=columns)
      # Standardize column names if 'key' and 'count' are present
      rename_map = {}
      if 'key' in df.columns:
        rename_map['key'] = 'Kind (or Aggregation Key)'
      if 'count' in df.columns:
        rename_map['count'] = 'Count'
      if rename_map:
          df = df.rename(columns=rename_map)
          # Sort if 'Kind' column exists
          if 'Kind (or Aggregation Key)' in df.columns:
             df.sort_values(by=['Kind (or Aggregation Key)'], inplace=True)
      # Print as Markdown table
      print(df.to_markdown(index=False))
    elif 'aggregations' in data and not data['aggregations']:
        print ("Query successful, but no aggregations returned.")
    else:
        print("Response does not contain 'aggregations' key or it's not a list.")
        print(json.dumps(data, indent=2))

  except requests.exceptions.RequestException as e:
    print(f"Error processing aggregation request: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")
  except Exception as e:
    print(f"An error occurred while processing aggregation results: {e}")

### 1. Basic Search (Query API)

Perform a simple search for Wellbore records (`osdu:wks:master-data--Wellbore:*`) using the `query` endpoint. We limit the results to 10.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the search payload
basic_query_payload = {
    "kind": "osdu:wks:master-data--Wellbore:*", # Search for any version of Wellbore
    "query": "*", # Match all records of the specified kind
    "limit": 10 # Return a maximum of 10 results
    # "returnedFields": ["id", "data.FacilityName"] # Optional: Specify which fields to return
}

print(f"Attempting basic search with payload:")
print(json.dumps(basic_query_payload, indent=2))

try:
    # Send POST request to the /query endpoint
    basic_search_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        data=json.dumps(basic_query_payload)
    )
    basic_search_response.raise_for_status() # Check for HTTP errors

    print(f"\nBasic search successful (Status Code: {basic_search_response.status_code}).")
    results_data = basic_search_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during basic search: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

### 2. Cursor Search (Query With Cursor API)

Use the `query_with_cursor` endpoint to retrieve large result sets page by page. The first request initiates the search and returns the first page along with a `cursor`. Subsequent requests include the `cursor` in the payload to fetch the next page.

**API Endpoint:** `POST /query_with_cursor`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
all_cursor_results = []
current_cursor = None
page_count = 0
max_pages_to_fetch = 3  # Limit the number of pages to fetch for this lab example
results_per_page = 5

print(f"Attempting cursor search for 'osdu:wks:master-data--Well:*' (Max {max_pages_to_fetch} pages, {results_per_page} results/page)")

try:
    while page_count < max_pages_to_fetch:
        page_count += 1
        print(f"\nFetching page {page_count}...")

        # Construct the payload for the current page
        cursor_payload = {
            "kind": "osdu:wks:master-data--Well:*", # Search for Well records
            "query": "*",
            "limit": results_per_page,
            "returnedFields": ["id", "data.FacilityName"] # Return only specific fields
        }
        # Include the cursor if it exists (for pages after the first)
        if current_cursor:
            cursor_payload["cursor"] = current_cursor
            print(f"   Using cursor: {current_cursor}")
        else:
            print("   No cursor provided (fetching first page).")

        # print(f"   Submitting payload: {json.dumps(cursor_payload, indent=2)}")

        # Send POST request to the /query_with_cursor endpoint
        cursor_response = requests.post(
            f"{search_endpoint}/query_with_cursor",
            headers=headers,
            json=cursor_payload # Use json= for automatic content-type header
        )
        cursor_response.raise_for_status()  # Raise exception for HTTP errors

        results_page_data = cursor_response.json()
        print(f"   Page {page_count} request successful (Status Code: {cursor_response.status_code}).")
        # print(f"Page {page_count} Raw Response:")
        print(json.dumps(results_page_data, indent=2))

        # Process results from the current page
        page_results = results_page_data.get('results', [])
        if page_results:
            print(f"   Found {len(page_results)} results on this page.")
            all_cursor_results.extend(page_results)
        else:
            print("   No more results found on this page.")
            break # Exit loop if no results are returned

        # Check for and update the cursor for the next iteration
        if 'cursor' in results_page_data and results_page_data['cursor']:
            current_cursor = results_page_data['cursor']
            print(f"   Received new cursor for next page: {current_cursor}")
        else:
            print("   No cursor returned in response. Ending search.")
            break # Exit loop if no cursor is provided

    print(f"\nCursor search finished after {page_count} page(s).")
    print(f"Total results fetched across all pages: {len(all_cursor_results)}")
    # Optionally print all fetched results
    print("All Fetched Results:")
    print(json.dumps(all_cursor_results, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during cursor search on page {page_count}: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

### 3. Text-based Search (Query API)

Use the standard `query` endpoint with Lucene syntax in the `query` parameter for more advanced text searching.

#### 3.1 Basic Text Search

Search across many kinds (`osdu:wks:*:*`) for records containing the text `"wellcom"` anywhere in their indexed fields. Text search is typically case-insensitive and matches parts of words.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the payload for a general text search
text_query_payload = {
    "kind": "osdu:wks:*:*",  # Search across many kinds within the 'wks' source
    "query": "D12",  # Lucene syntax: search for the term 'wellcom' in any indexed field
    "limit": 10
}

print(f"Attempting text search with payload:")
print(json.dumps(text_query_payload, indent=2))

try:
    text_search_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        data=json.dumps(text_query_payload)
    )
    text_search_response.raise_for_status()

    print(f"\nText search successful (Status Code: {text_search_response.status_code}).")
    results_data = text_search_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during text search: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

#### 3.2 Text Search in Specific Field

Search for records where the specific field `data.FacilityName` contains the text `"D12"`. Note the use of `FieldName:"Value"` Lucene syntax. This search will find records with facility names like `"Well D12"`, `"D12-Platform"`, `"D12-02"`, etc.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the payload for text search within a specific field
field_text_query_payload = {
    "kind": "osdu:wks:*:*",
    # Lucene query: Search for "D12" within the data.FacilityName field.
    # The backslashes escape the quotes within the JSON string.
    "query": "data.FacilityName:\"D12\"",
    "limit": 10,
    "returnedFields": ["id", "kind", "data.FacilityName"] # Return only relevant fields
}

print(f"Attempting field-specific text search with payload:")
print(json.dumps(field_text_query_payload, indent=2))

try:
    field_text_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        data=json.dumps(field_text_query_payload)
    )
    field_text_response.raise_for_status()

    print(f"\nField text search successful (Status Code: {field_text_response.status_code}).")
    results_data = field_text_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during field text search: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

#### 3.3 Exact Match Keyword Search

Search for records where the `data.FacilityName` field *exactly* matches `"D12"`. This uses the `.keyword` suffix (if configured in the search mapping for that field) to bypass text analysis and perform an exact, case-sensitive match.

*   Running with `"D12"` might return no results if records have names like `"D12-02"`.
*   Try changing the query value to `"D12-02"` (or another exact name found in the previous step) to see the difference.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the EXACT value to search for (try changing this after the first run)
exact_facility_name = "D12"
# exact_facility_name = "D12-02" # Example to try on second run

# Define the payload for keyword search (exact match)
keyword_query_payload = {
    "kind": "osdu:wks:*:*",
    # Lucene query: Search for the exact term using the .keyword field.
    # Note the triple backslashes needed: one pair for JSON string escaping of quotes,
    # and one for Lucene's requirement to escape the inner quotes of the exact phrase.
    "query": f"data.FacilityName.keyword:\"{exact_facility_name}\"",
    "limit": 10,
    "returnedFields": ["id", "kind", "data.FacilityName"]
}

print(f"Attempting keyword (exact match) search for FacilityName '{exact_facility_name}' with payload:")
print(json.dumps(keyword_query_payload, indent=2))

try:
    keyword_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        data=json.dumps(keyword_query_payload)
    )
    keyword_response.raise_for_status()

    print(f"\nKeyword search successful (Status Code: {keyword_response.status_code}).")
    results_data = keyword_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    if not results_data.get('results'):
        print(f"---> No records found with EXACT FacilityName '{exact_facility_name}'. This is expected.")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during keyword search: {e}")
    if e.response is not None:
        # A 400 Bad Request might occur if the .keyword field doesn't exist or the query syntax is wrong
        print(f"Response Body: {e.response.text}")

### 4. Advanced Search Features (Spatial)

#### 4.1 Spatial Search (By Distance)
Search for Wellbores within a specified distance (radius) of a central geographic point. This uses the `byDistance` spatial filter.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the spatial query payload (By Distance)
spatial_distance_payload = {
    "kind": "osdu:wks:master-data--Wellbore:1.0.0",
    "limit": 10,
    "query": "*", # Can be combined with other query criteria
    "spatialFilter": {
        "field": "data.SpatialLocation.Wgs84Coordinates", # The geo-point field in the schema to query against
        "byDistance": {
            "point": {
                "latitude": 52.780083,
                "longitude": 5.244389
            },
            "distance": 8000 # Distance in meters
        }
    },
    "returnedFields": ["id", "kind", "data.FacilityName", "data.SpatialLocation.Wgs84Coordinates"]
}

print(f"Attempting spatial search (By Distance) with payload:")
print(json.dumps(spatial_distance_payload, indent=2))

try:
    spatial_dist_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        json=spatial_distance_payload # Use json= for automatic content-type
    )
    spatial_dist_response.raise_for_status()

    print(f"\nSpatial (By Distance) search successful (Status Code: {spatial_dist_response.status_code}).")
    results_data = spatial_dist_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during spatial (By Distance) search: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

#### 4.2 Spatial Search (By Bounding Box)

Search for records within a rectangular area defined by top-left and bottom-right geographic coordinates using the `byBoundingBox` filter.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the spatial query payload (By Bounding Box)
spatial_bbox_payload = {
    "kind": "osdu:wks:master-data--Wellbore:1.0.0",
    "limit": 30,
    "query": "*",
    "spatialFilter": {
        "field": "data.SpatialLocation.Wgs84Coordinates",
        "byBoundingBox": {
            # Note: Ensure latitude/longitude order and range are correct for your CRS (usually WGS84)
            "topLeft": {"latitude": 52.326016, "longitude": 6.780958},
            "bottomRight": {"latitude": 51.863627, "longitude": 8.391779},
        }
    },
    "returnedFields": ["id", "kind", "data.FacilityName", "data.SpatialLocation.Wgs84Coordinates"]
}

print(f"Attempting spatial search (By Bounding Box) with payload:")
print(json.dumps(spatial_bbox_payload, indent=2))

try:
    spatial_bbox_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        json=spatial_bbox_payload
    )
    spatial_bbox_response.raise_for_status()

    print(f"\nSpatial (Bounding Box) search successful (Status Code: {spatial_bbox_response.status_code}).")
    results_data = spatial_bbox_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during spatial (Bounding Box) search: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

#### 4.3 Spatial Search (By GeoPolygon)

Search for records within an arbitrary polygon defined by a list of vertex coordinates using the `byGeoPolygon` filter. The first and last points in the list must be the same to close the polygon.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the spatial query payload (By GeoPolygon)
spatial_polygon_payload = {
  "kind": "osdu:wks:master-data--Wellbore:1.0.0",
  "limit": 30,
  "query": "*",
  "spatialFilter": {
    "field": "data.SpatialLocation.Wgs84Coordinates",
    "byGeoPolygon": {
      "points": [ # List of vertices defining the polygon
        {"latitude": 52.326016, "longitude": 6.780958},
        {"latitude": 51.863627, "longitude": 8.391779},
        {"latitude": 53.123515, "longitude": 7.504645},
        {"latitude": 52.326016, "longitude": 6.780958} # Close the polygon
      ]
    }
  },
  "returnedFields": ["id", "kind", "data.FacilityName", "data.SpatialLocation.Wgs84Coordinates"]
}

print(f"Attempting spatial search (By GeoPolygon) with payload:")
print(json.dumps(spatial_polygon_payload, indent=2))

try:
    spatial_poly_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        json=spatial_polygon_payload
    )
    spatial_poly_response.raise_for_status()

    print(f"\nSpatial (GeoPolygon) search successful (Status Code: {spatial_poly_response.status_code}).")
    results_data = spatial_poly_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results:")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during spatial (GeoPolygon) search: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

### 5. Sorting and Field Selection

Search for Well records, sort the results by their `id` (ascending), and return only the `id`, `kind`, and `data.FacilityName` fields.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` + ACL `VIEWER` access to result records.

In [ ]:
# Define the query payload with sorting and field selection
sorted_query_payload = {
  "kind": "osdu:wks:master-data--Well:1.0.0",
  "query": "*",
  "offset": 0, # Start from the first result
  "limit": 30,
  "sort": {
    "field": ["id"], # Field(s) to sort by (use 'id.keyword' or 'data.FacilityName.keyword' for exact string sorting if available)
    "order": ["ASC"] # Sort order: ASC or DESC
  },
  "returnedFields": [ "id", "kind", "data.FacilityName" ] # Specify fields to return
}

print(f"Attempting sorted search with payload:")
print(json.dumps(sorted_query_payload, indent=2))

try:
    sorted_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        json=sorted_query_payload
    )
    sorted_response.raise_for_status()

    print(f"\nSorted Query successful (Status Code: {sorted_response.status_code}).")
    results_data = sorted_response.json()
    print(f"Total results matching query (if available): {results_data.get('totalCount', 'N/A')}")
    print(f"Number of results returned: {len(results_data.get('results', []))}")
    print("Query Results (Sorted by ID):")
    print(json.dumps(results_data, indent=2))

except requests.exceptions.RequestException as e:
    print(f"\nError during sorted query: {e}")
    if e.response is not None:
        # 400 Bad Request might occur if sort field doesn't exist or isn't sortable
        print(f"Response Body: {e.response.text}")

### 6. Aggregated Search

Perform a search that aggregates results based on a specified field. This example counts the number of records for each `kind` present in the data partition.

**API Endpoint:** `POST /query`
**Permissions:** `service.search.user` (ACLs not directly relevant for aggregations, only for the base query matching).

In [ ]:
# Define the aggregation query payload
aggregate_query_payload = {
    "kind": "*:*:*:*", # Aggregate across all kinds in the partition
    "limit": 0, # We don't need individual results, only the aggregation
    "query": "*", # Match all documents
    "aggregateBy": "kind" # Field to aggregate on
}

print(f"Attempting aggregation search by 'kind' with payload:")
print(json.dumps(aggregate_query_payload, indent=2))

try:
    aggregate_response = requests.post(
        f"{search_endpoint}/query",
        json=aggregate_query_payload,
        headers=headers
    )
    # Use the helper function to process and print the result
    print(f"\nAggregation search status code: {aggregate_response.status_code}")
    print("Aggregation Results (Count per Kind):")
    print_agg_list_result(aggregate_response)

except requests.exceptions.RequestException as e:
    # Error handling specifically for the request sending part
    print(f"\nError during aggregation query request: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

### 7. Challenge

**Tasks:**
1.  **Count Wellbores:** How many `master-data--Wellbore` records (any version) exist in the data partition?
2.  **Find Deepest Trajectory:** Find the `work-product-component--WellboreTrajectory` record with the largest measured depth (`data.BaseDepthMeasuredDepth`). Return the `id` of this trajectory record, its `data.WellboreID` (the parent wellbore it belongs to), and its `data.BaseDepthMeasuredDepth` value.

**Hints:**
*   For counting, use `limit: 0` and `trackTotalCount: true` or `aggregateBy` if appropriate.
*   For finding the deepest, use the `sort` parameter on the `data.BaseDepthMeasuredDepth` field in descending order (`DESC`) and set `limit: 1`.
*   Target the correct `kind` for each task (`osdu:wks:master-data--Wellbore:*` for counting, `osdu:wks:work-product-component--WellboreTrajectory:*` for deepest).
*   Ensure you request the necessary fields using `returnedFields`.

In [ ]:
# Code block to run the challenge
print("--- Starting Search Service Challenge ---")

# Use standard 'headers' and 'search_endpoint'

#### 💡 Solution Code Cells

##### 1. Count Wellbores

In [ ]:
print("\nChallenge Part 1: Counting Wellbores...")

count_wellbores_payload = {
  "kind": "osdu:wks:master-data--Wellbore:*", # Target Wellbore kind
  "query": "*",
  "limit": 0,           # We only need the count, not the results themselves
  "trackTotalCount": True # Explicitly request the total count
}

print(f"   Request Body: {json.dumps(count_wellbores_payload, indent=2)}")

try:
    count_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        json=count_wellbores_payload
    )
    count_response.raise_for_status()
    count_data = count_response.json()

    print(f"   Count request successful ({count_response.status_code}).")
    total_wellbores = count_data.get('totalCount', 'Not Available')
    print(f"   >>> Total number of Wellbore records found: {total_wellbores}")

except requests.exceptions.RequestException as e:
    print(f"   Error counting wellbores: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")

##### 2. Find Deepest Trajectory

In [ ]:
print("\nChallenge Part 2: Finding the deepest WellboreTrajectory...")

deepest_trajectory_payload = {
  "kind": "osdu:wks:work-product-component--WellboreTrajectory:*", # Target Trajectory kind
  "query": "*", # Consider adding 'data.BaseDepthMeasuredDepth:[0 TO *]' if 0 or negative depths are invalid
  "limit": 1,   # We only need the top result
  "sort": {
    "field": ["data.BaseDepthMeasuredDepth"], # Field to sort by
    "order": ["DESC"] # Descending order to get the largest value first
  },
  "returnedFields": [ "id", "data.WellboreID", "data.BaseDepthMeasuredDepth" ] # Fields needed
}

print(f"   Request Body: {json.dumps(deepest_trajectory_payload, indent=2)}")

try:
    deepest_response = requests.post(
        f"{search_endpoint}/query",
        headers=headers,
        json=deepest_trajectory_payload
    )
    deepest_response.raise_for_status()
    deepest_data = deepest_response.json()

    print(f"   Deepest trajectory search successful ({deepest_response.status_code}).")

    if 'results' in deepest_data and deepest_data['results']:
        deepest_record = deepest_data['results'][0]
        print("   >>> Deepest WellboreTrajectory Found:")
        print(f"       Record ID: {deepest_record.get('id')}")
        print(f"       Parent Wellbore ID: {deepest_record.get('data', {}).get('WellboreID')}")
        print(f"       Base Measured Depth: {deepest_record.get('data', {}).get('BaseDepthMeasuredDepth')}")
        # Optionally print the whole record:
        # print(json.dumps(deepest_record, indent=4))
    else:
        print("   >>> No WellboreTrajectory records found matching the criteria.")

except requests.exceptions.RequestException as e:
    print(f"   Error finding deepest trajectory: {e}")
    if e.response is not None:
        # 400 might occur if sort field is not sortable
        print(f"   Response Body: {e.response.text}")

print("--- Search Service Challenge Complete ---")

## OSDU Storage & Dataset Service Lab

This lab exercise guides you through interacting with the OSDU Storage and Dataset services. You will:
1.  Obtain instructions (like a signed URL) for uploading a file.
2.  Upload a file (an image) to the location specified by the instructions.
3.  Register the uploaded file as a Dataset record in OSDU, linking it to the file's location and adding metadata (like legal tags and ACLs).
4.  Manage versions of the Dataset record.
5.  Retrieve download instructions for the file.
6.  Download and display the file.
7.  Perform soft delete and purge operations on the record.

**Key Concepts:**
*   **Storage Service:** Manages metadata records (like wells, logs, documents). Handles record creation, versioning, retrieval, deletion.
*   **Dataset Service:** Facilitates interaction with bulk data associated with OSDU records (e.g., files, datasets). Provides mechanisms to get upload/download instructions (URLs) for cloud storage.
*   **Signed URL:** A temporary, secure URL granting time-limited access to a specific cloud storage object (e.g., for upload or download).
*   **FileSource / Dataset Registry:** Information within an OSDU record pointing to the actual location of the associated file/data in cloud storage.

**Permissions Note:** Operations typically require combinations of `service.storage.*` and `service.dataset.*` roles (creator, editor, viewer, admin), plus `service.legal.viewer` for associating legal tags.

**Documentation:**
*   [Storage API](https://osdu.pages.opengroup.org/platform/system/storage/)
*   [Dataset API](https://osdu.pages.opengroup.org/platform/system/dataset/)

### Setup: Storage & Dataset Lab

This cell sets up necessary variables and performs prerequisite actions (like creating a legal tag) for the Storage & Dataset lab.

**Dependencies:**
*   Requires `requests`, `json`, `os`, `IPython.display.Image`.
*   Relies on variables from initial authentication: `osdu_endpoint`, `headers`, `user_id`, `osdu_data_partition_id`, `entitlements_domain`, `legal_endpoint`.
*   Requires the `./assets/osdu.png` file to exist relative to the notebook's location.

In [ ]:
import os # Needed for path manipulation
import requests
import json
# updated colab
# from IPython.display import Image # To display the image later
from PIL import Image

# Define API Endpoints
dataset_endpoint = f"{osdu_endpoint}/api/dataset/v1"
storage_endpoint = f"{osdu_endpoint}/api/storage/v2"
file_endpoint = f"{osdu_endpoint}/api/file/v2"
# legal_endpoint is assumed to be defined from the Legal Lab setup
if 'legal_endpoint' not in locals():
    legal_endpoint = f"{osdu_endpoint}/api/legal/v1"
print(f"Using Dataset API Endpoint: {dataset_endpoint}")
print(f"Using Storage API Endpoint: {storage_endpoint}")

# updated / commented for Colab
# Define the file to be uploaded
# file_path = "./assets/osdu.png" # Relative path to assets directory
# file_name = os.path.basename(file_path) # Extract the file name from the path
# print(f"Using file path: {file_path}")
# if not os.path.exists(file_path):
#      print(f"\n*** WARNING: File '{file_path}' not found. Please ensure it exists. ***\n")
file_name = "osdu.png" # Extract the file name from the path

# --- Action: Create a Legal Tag for this Lab ---
# This tag is needed to associate with the record we create.
lab_legal_tag_name = f'{osdu_data_partition_id}-storage-lab-tag-{user_id}'
lab_legal_tag_description = f"Storage/Dataset Lab tag for user {user_id}"
lab_legal_tag_payload = {
    "name": lab_legal_tag_name,
    "description": lab_legal_tag_description,
    "properties": { # Ensure these match required fields and reference values
        "contractId": f"LabContract-{user_id}",
        "countryOfOrigin": ["US"],
        "expirationDate": "2099-12-31",
        "originator": f"StorageLab-{user_id}",
        "dataType": "Public Domain Data",
        "securityClassification": "Public",
        "personalData": "No Personal Data",
        "exportClassification": "EAR99"
    }
}
tag_name_for_record = None # Initialize variable
try:
    print(f"Attempting to create/ensure legal tag: {lab_legal_tag_name}")
    tag_create_response = requests.post(
        f"{legal_endpoint}/legaltags",
        headers=headers,
        data=json.dumps(lab_legal_tag_payload)
    )
    if tag_create_response.status_code == 409:
        print(f"   Legal tag '{lab_legal_tag_name}' already exists (409). Using existing tag.")
        tag_name_for_record = lab_legal_tag_name
    else:
        tag_create_response.raise_for_status()
        tag_name_for_record = tag_create_response.json().get("name", lab_legal_tag_name)
        print(f"   Legal tag '{tag_name_for_record}' created successfully ({tag_create_response.status_code}).")
except requests.exceptions.RequestException as e:
    print(f"   Error creating/ensuring legal tag: {e}")
    if e.response is not None:
        print(f"   Response Body: {e.response.text}")
    print("   *** WARNING: Failed to create/find legal tag. Record creation in Step 3 might fail. ***")

# Variables to store results from steps
upload_signed_url = None
upload_file_source = None
created_record_id = None
created_record_version = None

### 1. Get Upload Instructions (Signed URL)

Request instructions from the Dataset service for uploading a file. This typically returns provider-specific information, including a temporary, secure URL (`signedUrl`) where the file can be uploaded and a `fileSource` identifier to be stored in the OSDU record.

**API Endpoint:** `POST /storageInstructions`
**Permissions:** `service.dataset.creator`

In [ ]:
# Define the kindSubtype for the file we intend to upload
# This helps the service determine appropriate storage, but is often generic
kind_subtype = "dataset--File.Generic" # Or more specific like "dataset--File.Image"
expiry_time = "2h"  # How long the upload URL should be valid (e.g., 2 hours)

print(f"Requesting upload instructions for kindSubtype: {kind_subtype}")

try:
    # Send POST request to get storage instructions
    instructions_response = requests.post(
        f"{dataset_endpoint}/storageInstructions?kindSubType={kind_subtype}&expiryTime={expiry_time}",
        headers=headers
        # No request body needed for this endpoint
    )
    instructions_response.raise_for_status() # Check for HTTP errors

    instructions_data = instructions_response.json()
    print(f"Upload instructions received successfully (Status Code: {instructions_response.status_code}).")
    print("Instructions Response:")
    print(json.dumps(instructions_data, indent=2))

    # Extract the signedUrl and fileSource from the response
    # Structure might vary slightly by provider ('providerKey' vs 'fileSource')
    storage_location = instructions_data.get("storageLocation", {})
    upload_signed_url = storage_location.get("signedUrl")
    upload_file_source = storage_location.get("fileSource", storage_location.get("providerKey")) # Adapt as needed

    if upload_signed_url and upload_file_source:
        print(f"\nExtracted Signed URL: {upload_signed_url[:100]}...") # Print beginning of URL
        print(f"Extracted File Source/Provider Key: {upload_file_source}")
    else:
        print("\n*** WARNING: Could not extract 'signedUrl' or 'fileSource'/'providerKey' from the response. ***")

except requests.exceptions.RequestException as e:
    print(f"\nError getting upload instructions: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

### 2. Upload File Data

Upload the actual file content (`osdu.png`) to the `signedUrl` obtained in the previous step.

**API Endpoint:** The `signedUrl` itself (external cloud provider endpoint)
**Permissions:** Granted by the temporary token embedded in the `signedUrl`.

In [ ]:
# Check if we have the signed URL and the file exists
#commented for Colab
# if upload_signed_url and os.path.exists(file_path):
#     print(f"Attempting to upload file '{file_path}' to the signed URL...")

# Prepare headers for the PUT request to the cloud storage URL
# Content-Type is important for how the file is stored and later retrieved
upload_headers = {
    "Content-Type": "text/plain", # Specific content type
    "x-ms-blob-type": "BlockBlob",  # Adjust based on the file type
    "data-partition-id": osdu_data_partition_id  # Get the file size
}
print(f"   Upload Headers: {upload_headers}")

try:
    # # Open the file in binary read mode ('rb')
    # with open(file_path, "rb") as file_data:
    #     # Send PUT request with file data as the body
    #     upload_response = requests.put(
    #         upload_signed_url,
    #         headers=upload_headers,
    #         data=file_data
    #     )

    # Colab version
    file_data = get_bytes("osdu.png")
    # Send PUT request with file data as the body
    upload_response = requests.put(
        upload_signed_url,
        headers=upload_headers,
        data=file_data
    )

    # Check for success (usually 201 Created for Azure Blob, might be 200 OK for others)
    if upload_response.status_code == 201 or upload_response.status_code == 200:
        print("Upload Response:")
        print("The file is uploaded.")
    else:
        # Raise an exception for other non-successful statuses
        upload_response.raise_for_status()

except requests.exceptions.RequestException as e:
    print(f"\nError uploading file: {e}")
    # The response body from the cloud provider might contain useful error details
    if e.response is not None:
        print(f"Response Status Code: {e.response.status_code}")
        print(f"Response Body: {e.response.text}")

#commented for Colab
# elif not upload_signed_url:
#     print("Skipping file upload because 'upload_signed_url' was not obtained in Step 1.")
# else:
#      print(f"Skipping file upload because file '{file_path}' does not exist.")

### 3. Create Dataset Record

Register the uploaded file in OSDU by creating a `dataset` record. This record includes:
*   A unique OSDU ID.
*   The `kind` specifying the type of dataset.
*   ACLs defining access permissions.
*   Legal tags for compliance.
*   Crucially, the `DatasetProperties.FileSourceInfo` section containing the `FileSource` (or `providerKey`) obtained in Step 1, linking this OSDU record to the actual file in cloud storage.

**API Endpoint:** `PUT /registerDataset`
**Permissions:** `service.dataset.creator`, `service.storage.creator`, `service.legal.viewer`

In [ ]:
# Check if prerequisite steps were successful
if upload_file_source and tag_name_for_record:
    # Define the record ID (must be unique)
    record_id = f"{osdu_data_partition_id}:dataset--File.Generic:{user_id}"
    print(f"Attempting to register dataset with ID: {record_id}")

    # Define the payload for the dataset record
    # Note: The endpoint expects a list of records
    record_registration_payload = {
      "datasetRegistries": [
        {
            "id": record_id,
            "kind": "osdu:wks:dataset--File.Generic:1.0.0", # Match the schema kind
            "acl": {
                "owners": [f"data.default.owners@{entitlements_domain}"],
                "viewers": [f"data.default.viewers@{entitlements_domain}"],
            },
            "legal": {
                "legaltags": [tag_name_for_record], # Use the tag created in setup
                "otherRelevantDataCountries": ["US"], # Example
            },
            "data": {
                "Name": file_name, # Original file name
                "Description": f"OSDU PNG Image uploaded by user {user_id}",
                "DatasetProperties": {
                    "FileSourceInfo": {
                        "FileSource": upload_file_source, # The crucial link from Step 1
                        "Name": file_name # Can be repeated here
                        # "PreloadFilePath": "..." # Optional, may be needed by some DDMS
                    }
                },
                "SchemaFormatTypeID": f"{osdu_data_partition_id}:reference-data--SchemaFormatType:Image.PNG:" # Optional: Ref data link for file format
            }
        }
      ]
    }

    print(f"   Registration Payload: {json.dumps(record_registration_payload, indent=2)}")

    try:
        # Send PUT request to register the dataset
        register_response = requests.put(
            f"{dataset_endpoint}/registerDataset",
            headers=headers,
            json=record_registration_payload # Use json= for automatic content-type
        )
        register_response.raise_for_status() # Check for HTTP errors

        print(f"Dataset registration successful (Status Code: {register_response.status_code}).")
        record_response_data = register_response.json()
        print("Registration Response:")
        print(json.dumps(record_response_data, indent=2))

        # Extract the created record ID and version (important for later steps)
        if record_response_data.get("datasetRegistries"):
             created_record_id = record_response_data["datasetRegistries"][0]["id"]
             print(f"   Stored created_record_id: {created_record_id}")
        else:
             # Fallback if recordIds isn't present but request succeeded
             created_record_id = record_id
             print(f"   Could not find 'recordIds' in response, using provided ID: {created_record_id}")

        if record_response_data.get("datasetRegistries"):
             created_record_version = record_response_data["datasetRegistries"][0]["version"]
             print(f"   Stored created_record_version: {created_record_version}")
        else:
             print("   Could not find 'version' in response.")

    except requests.exceptions.RequestException as e:
        print(f"\nError registering dataset record: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Skipping dataset registration because 'upload_file_source' or 'tag_name_for_record' was not set in previous steps.")

### 4. Get All Record Versions

List all available versions for the dataset record created in the previous step.

**API Endpoint:** `GET /records/versions/{recordId}`
**Permissions:** `service.storage.viewer`

In [ ]:
# Ensure created_record_id is set
if created_record_id:
    print(f"Attempting to get all versions for record: {created_record_id}")
    try:
        # Send GET request to the record versions endpoint
        versions_response = requests.get(
             f"{storage_endpoint}/records/versions/{created_record_id}",
             headers=headers
        )
        versions_response.raise_for_status() # Check for HTTP errors (e.g., 404 if record not found)

        print(f"Get versions successful (Status Code: {versions_response.status_code}).")
        versions_data = versions_response.json()
        print("Available Versions:")
        print(json.dumps(versions_data, indent=2))

        # Optionally, store the latest version number if needed elsewhere
        # latest_version = versions_data.get("versions", [None])[-1]
        # print(f"Latest version number: {latest_version}")

    except requests.exceptions.RequestException as e:
        print(f"\nError getting record versions: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Skipping get versions because 'created_record_id' was not set in Step 3.")

### 5. Get Specific Record Version

Retrieve the specific version of the record that was created in Step 3, using the `created_record_version` variable captured then.

**API Endpoint:** `GET /records/{recordId}/{version}`
**Permissions:** `service.storage.viewer`

In [ ]:
# Ensure we have both the record ID and the specific version number
if created_record_id and created_record_version:
    print(f"Attempting to get specific version {created_record_version} for record: {created_record_id}")
    try:
        # Send GET request for the specific record version
        specific_version_response = requests.get(
            f"{storage_endpoint}/records/{created_record_id}/{created_record_version}",
            headers=headers
        )
        specific_version_response.raise_for_status() # Check for errors

        print(f"Get specific version successful (Status Code: {specific_version_response.status_code}).")
        print("Record Version Data:")
        print(json.dumps(specific_version_response.json(), indent=2))

    except requests.exceptions.RequestException as e:
        print(f"\nError getting specific record version: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
elif not created_record_id:
    print("Skipping get specific version because 'created_record_id' was not set in Step 3.")
else:
     print("Skipping get specific version because 'created_record_version' was not captured in Step 3.")

### 6. Get Latest Record Version

Retrieve the *latest* version of the record. This is done by omitting the version number from the URL. The Storage service returns the version with the highest version number.

**API Endpoint:** `GET /records/{recordId}`
**Permissions:** `service.storage.viewer`

In [ ]:
# Ensure created_record_id is set
if created_record_id:
    print(f"Attempting to get the LATEST version for record: {created_record_id}")
    try:
        # Send GET request without a version number
        latest_version_response = requests.get(
            f"{storage_endpoint}/records/{created_record_id}",
            headers=headers
        )
        latest_version_response.raise_for_status() # Check for errors

        print(f"Get latest version successful (Status Code: {latest_version_response.status_code}).")
        print("Latest Record Version Data:")
        latest_data = latest_version_response.json()
        print(json.dumps(latest_data, indent=2))
        # You can compare latest_data['version'] with created_record_version
        # print(f"Latest version found: {latest_data.get('version')}")

    except requests.exceptions.RequestException as e:
        print(f"\nError getting latest record version: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Skipping get latest version because 'created_record_id' was not set in Step 3.")

### 7. Get Download Instructions & Display Image

Request download instructions (another signed URL) for the dataset record, then use that URL to fetch the image data and display it.

**API Endpoint:** `GET /retrievalInstructions`
**Permissions:** `service.dataset.viewer`

In [ ]:
from io import BytesIO
from IPython.display import display

# Ensure created_record_id is set
if created_record_id:
    print(f"Requesting download instructions for record: {created_record_id}")
    download_signed_url = None # Initialize variable
    try:
        # Send GET request for retrieval instructions
        download_instructions_response = requests.get(
            f"{dataset_endpoint}/retrievalInstructions?id={created_record_id}&expiryTime={expiry_time}", # Pass record ID as query parameter
            headers=headers
        )
        download_instructions_response.raise_for_status()

        download_instructions_data = download_instructions_response.json()
        print(f"Download instructions received successfully (Status Code: {download_instructions_response.status_code}).")
        print("Instructions Response:")
        print(json.dumps(download_instructions_data, indent=2))

        # Extract the signed URL (structure can be nested)
        datasets_list = download_instructions_data.get("datasets", [])
        if datasets_list and isinstance(datasets_list, list):
            # Assuming we want the URL for the first (and likely only) dataset in the list
            first_dataset = datasets_list[0]
            retrieval_properties = first_dataset.get("retrievalProperties", {})
            download_signed_url = retrieval_properties.get("signedUrl")

        if download_signed_url:
            print(f"\nExtracted Download Signed URL: {download_signed_url[:100]}...")

            # --- Download and Display Image ---
            print(f"\nAttempting to download image from signed URL...")
            try:
                # Send GET request to the download URL itself
                image_download_response = requests.get(download_signed_url)
                image_download_response.raise_for_status()
                print(f"   Image download successful (Status Code: {image_download_response.status_code}).")

                # Display the image directly using IPython.display.Image
                # This fetches the image data using the URL provided
                print("   Displaying image:")

                # updated colab
                #display(Image(url=download_signed_url))
                response = requests.get(download_signed_url)
                # print(response.content)

                display(Image.open(BytesIO(response.content)))

            except requests.exceptions.RequestException as img_e:
                print(f"   Error downloading image from signed URL: {img_e}")
                if img_e.response is not None:
                    print(f"   Response Status Code: {img_e.response.status_code}")
                    # Avoid printing large binary content for image errors
                    print(f"   Response Headers: {img_e.response.headers}")
            except Exception as display_e:
                 print(f"   Error displaying image: {display_e}")
        else:
            print("\n*** WARNING: Could not extract download 'signedUrl' from the retrieval instructions. ***")

    except requests.exceptions.RequestException as e:
        print(f"\nError getting download instructions: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Skipping download because 'created_record_id' was not set in Step 3.")

### 8. Soft Delete Record

Mark the record for deletion. The record still exists in the system and can potentially be recovered (e.g., by creating a new version), but it will typically be excluded from standard search results.

**API Endpoint:** `DELETE /records/{recordId}`
**Permissions:** `service.storage.editor`

In [ ]:
from io import BytesIO
from IPython.display import display

# Ensure created_record_id is set
if created_record_id:
    print(f"Attempting to SOFT delete record: {created_record_id}")
    try:
        # Send DELETE request to the record ID endpoint
        soft_delete_response = requests.delete(
            f"{storage_endpoint}/records/{created_record_id}",
            headers=headers
        )
        soft_delete_response.raise_for_status() # Should return 204 No Content on success

        print(f"Record soft deleted successfully (Status Code: {soft_delete_response.status_code}).")
        # Response body is typically empty
        if soft_delete_response.text:
             print(f"Response Body: {soft_delete_response.text}")

        # Optional: Verify soft delete by trying to get the latest version (should fail or return specific status)
        # Some systems might return 404, others might return the record with a 'deleted' status marker.
        print("\n   Verifying soft delete by attempting to get latest record...")
        try:
            verify_get = requests.get(f"{storage_endpoint}/records/{created_record_id}", headers=headers)
            if verify_get.status_code == 404:
                print("   Verification: Get request failed with 404 Not Found (expected after soft delete)." )
            else:
                verify_get.raise_for_status()
                print(f"   Verification Warning: Get request returned {verify_get.status_code}. Record might still be accessible or marked differently.")
                print(f"   Response: {json.dumps(verify_get.json(), indent=2)}")
        except requests.exceptions.RequestException as ve:
             if ve.response is not None and ve.response.status_code == 404:
                  print("   Verification: Get request failed with 404 Not Found (expected after soft delete)." )
             else:
                  print(f"   Verification Error: {ve}")

    except requests.exceptions.RequestException as e:
        print(f"\nError soft deleting record: {e}")
        if e.response is not None:
             # 404 here might mean it was already deleted
            print(f"Response Body: {e.response.text}")
else:
    print("Skipping soft delete because 'created_record_id' was not set.")

### 9. Create New Version (Undelete)

Creating a new version of a soft-deleted record effectively undeletes it. We use the same `registerDataset` endpoint as in Step 3, providing the same record ID. This creates a new version entry and makes the record active again.

**API Endpoint:** `PUT /registerDataset`
**Permissions:** `service.dataset.creator`, `service.storage.creator`, `service.legal.viewer`

In [ ]:
# Check if we have the necessary information to re-create the record
if created_record_id and upload_file_source and tag_name_for_record:
    print(f"Attempting to create a new version (undelete) for record: {created_record_id}")

    # Reuse or reconstruct the payload from Step 3
    # (Could modify description, etc., to indicate it's a new version)
    undelete_payload = {
      "datasetRegistries": [
        {
            "id": created_record_id, # Use the SAME ID
            "kind": "osdu:wks:dataset--File.Generic:1.0.0",
            "acl": {
                "owners": [f"data.default.owners@{entitlements_domain}"],
                "viewers": [f"data.default.viewers@{entitlements_domain}"],
            },
            "legal": {
                "legaltags": [tag_name_for_record],
                "otherRelevantDataCountries": ["US"],
            },
            "data": {
                "Name": file_name,
                "Description": f"OSDU PNG Image - Restored Version by user {user_id}", # Updated description
                "DatasetProperties": {
                    "FileSourceInfo": {
                        "FileSource": upload_file_source,
                        "Name": file_name
                    }
                },
                "SchemaFormatTypeID": f"{osdu_data_partition_id}:reference-data--SchemaFormatType:Image.PNG:",
            }
        }
      ]
    }

    print(f"   Update/Undelete Payload: {json.dumps(undelete_payload, indent=2)}")

    try:
        # Send PUT request to registerDataset again with the same ID
        undelete_response = requests.put(
            f"{dataset_endpoint}/registerDataset",
            headers=headers,
            json=undelete_payload
        )
        undelete_response.raise_for_status()

        print(f"New version creation (undelete) successful (Status Code: {undelete_response.status_code}).")
        undelete_response_data = undelete_response.json()
        print("Response:")
        print(json.dumps(undelete_response_data, indent=2))

        # Capture the NEW version number
        if undelete_response_data.get("recordVersions"):
             new_version = undelete_response_data["recordVersions"][0]
             print(f"   New record version created: {new_version}")
             # Update the global variable if needed for subsequent steps
             created_record_version = new_version
        else:
             print("   Could not find new version number in response.")

    except requests.exceptions.RequestException as e:
        print(f"\nError creating new version (undeleting) record: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("Skipping undelete because necessary information (record ID, file source, tag name) is missing.")

### 10. Purge Record

Permanently delete the record and all its versions. This action is irreversible and typically requires administrative privileges. The associated physical file in cloud storage may or may not be deleted depending on the platform's configuration.

**API Endpoint:** `POST /records:delete` (Note: V2 endpoint)
**Permissions:** `service.storage.admin`

Links:
- [Swagger](https://community.opengroup.org/osdu/platform/system/storage/-/blob/master/docs/api/storage_openapi.yaml?ref_type=heads)

In [ ]:
# Ensure created_record_id is set
if created_record_id:
    print(f"Attempting to PURGE record (irreversible): {created_record_id}")

    # Attempt to delete the dataset record using the registeredDataset endpoint.
    if created_record_id:
        print(f"\nAttempting to delete dataset using 'registeredDataset' API for record: {created_record_id}")
        try:
            delete_url = f"{file_endpoint}/files/{created_record_id}/metadata"
            # No additional payload is required for this request.
            response = requests.delete(delete_url, headers=headers)
            response.raise_for_status()
            print(f"Dataset delete successful (Status Code: {response.status_code}).")
            if response.text:
                print("Response Body:")
                print(response.text)
        except requests.exceptions.RequestException as e:
            print(f"Error during dataset delete: {e}")
            if e.response is not None:
                print(f"Response Body: {e.response.text}")
    else:
        print("No dataset record ID found, skipping dataset soft delete.")

    # The V2 purge endpoint expects a POST request with the record ID(s) in the body
    purge_payload = {
        "recordIds": [created_record_id]
    }
    print(f"   Purge Request Body: {json.dumps(purge_payload)}")
    print("   --- NOTE: This operation requires admin permissions and is permanent. ---")

    try:
        # Send POST request to the :delete endpoint
        purge_response = requests.post(
            f"{storage_endpoint}/records/delete", # Note the :delete action endpoint
            headers=headers,
            json=[created_record_id]
        )
        purge_response.raise_for_status() # Check for HTTP errors (e.g., 403 Forbidden)

        # Successful purge might return 200 OK or 204 No Content depending on implementation
        print(f"Record purge request successful (Status Code: {purge_response.status_code}).")
        # Response body might be empty or confirm deletion
        if purge_response.text and purge_response.status_code != 204:
             print(f"Response Body: {json.dumps(purge_response.json(), indent=2)}")
        else:
             print("Response Body: (empty or No Content)")

        # Clean up the variable as the record is gone
        print(f"\nDeleting variable created_record_id ('{created_record_id}') from notebook memory.")
        del created_record_id
        if 'created_record_version' in locals(): del created_record_version

    except requests.exceptions.RequestException as e:
        print(f"\nError purging record: {e}")
        if e.response is not None:
            print(f"Response Status Code: {e.response.status_code}")
            print(f"Response Body: {e.response.text}")
            if e.response.status_code == 403:
                 print("   Hint: Status code 403 (Forbidden) usually indicates missing 'service.storage.admin' permissions for purge.")
            elif e.response.status_code == 404:
                 print("   Hint: Status code 404 (Not Found) might indicate the record was already purged.")

    except NameError:
         # Handle case where variable was already deleted
        print("Variable 'created_record_id' no longer exists (likely already purged). ")
else:
    print("Skipping purge because 'created_record_id' was not set or already deleted.")

### Challenge

1.  **Modify Step 3 (Create Dataset Record):**
    *   Run the code cell for Step 3 once to create the initial record and capture its `id` and `version` (let's call this `version1`).
    *   Modify the `Description` field in the payload within the Step 3 code cell (e.g., add " - Second Version" to the description).
    *   Run the Step 3 code cell *again* with the modified payload (using the *same* `record_id`). This will create a second version. Capture the new version number returned in the response (let's call this `version2`).
2.  **Verify Versions:** Run Step 4 (Get All Record Versions) to confirm that the record now has two versions listed.
3.  **Retrieve First Version:** Use the `recordId` and the *first* version number (`version1`) you captured to run Step 5 (Get Specific Record Version) and display the details of the original record.
4.  **(Optional Cleanup):** Run Steps 8 (Soft Delete) and 10 (Purge Record) to remove the record created during the challenge.

# Lab 2: Wellbore DDMS Operations

**Goal:** Interact with the OSDU Wellbore Domain Data Management Services (DDMS) to manage wellbore trajectory data.

**Concepts Covered:**
*   Creating prerequisite resources (Legal Tag, Wellbore master record).
*   Creating a WellboreTrajectory Work Product Component (WPC).
*   Uploading trajectory data (Parquet format) via the Wellbore DDMS.
*   Retrieving trajectory metadata and data.
*   Visualizing trajectory data.
*   Cleaning up created resources.

## Setup for Lab 2

This section imports necessary libraries and sets up resources required for the Wellbore DDMS exercises:
*   **API Endpoints:** Defines the URLs for Wellbore DDMS, Storage, and Legal services.
*   **Unique Names:** Creates unique names for the Wellbore and Legal Tag using the `user_id`.
*   **Legal Tag:** Creates a legal tag required for associating with OSDU records.
*   **Placeholder Wellbore:** Creates a master-data Wellbore record. DDMS objects (like Trajectory) need to reference an existing Wellbore record.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from io import BytesIO
import os # For path manipulation

# Ensure setup ran correctly
if not all([osdu_endpoint, osdu_data_partition_id, user_id, headers, entitlements_domain]):
     raise ValueError("Initial setup failed or variables are missing. Cannot proceed with Lab 2.")

# --- API Endpoints ---
wellbore_dms_endpoint = f"{osdu_endpoint}/api/os-wellbore-ddms/ddms/v3"
storage_endpoint = f"{osdu_endpoint}/api/storage/v2"
legal_endpoint = f"{osdu_endpoint}/api/legal/v1"
print(f"Wellbore DMS Endpoint: {wellbore_dms_endpoint}")
print(f"Storage Endpoint: {storage_endpoint}")
print(f"Legal Endpoint: {legal_endpoint}")

# --- Unique Resource Identifiers ---
legal_tag_name = f'osdu-lab-tag-{user_id}'
wellbore_name = f"LabWellbore-{user_id}"
# Construct the preliminary Wellbore ID (the final ID might be slightly different after creation)
wellbore_id_prefix = f"{osdu_data_partition_id}:master-data--Wellbore"
wellbore_id_initial = f"{wellbore_id_prefix}:{wellbore_name}:"
wellbore_id = None # Will be set after successful creation
print(f"Will attempt to use Legal Tag: {legal_tag_name}")
print(f"Will attempt to create Wellbore: {wellbore_id_initial}")

# --- Create Legal Tag ---
# Define the legal tag payload
legal_tag_payload = {
    "name": legal_tag_name,
    "description": f"Legal tag created for OSDU lab session user {user_id}",
    "properties": {
        "contractId": f"LabContract-{user_id}",
        "countryOfOrigin": ["US"], # Example value
        "expirationDate": "2099-12-31",
        "originator": f"LabUser-{user_id}",
        "dataType": "Public Domain Data", # Example value
        "securityClassification": "Public", # Example value
        "personalData": "No Personal Data",
        "exportClassification": "EAR99" # Example value
    }
}

try:
    print(f"\nAttempting to create Legal Tag: {legal_tag_name}...")
    tag_response = requests.post(
        f"{legal_endpoint}/legaltags",
        headers=headers,
        data=json.dumps(legal_tag_payload)
    )

    if tag_response.status_code == 201:
        created_tag_name = tag_response.json().get("name")
        print(f"Successfully created Legal Tag: {created_tag_name}")
    elif tag_response.status_code == 409:
        print(f"Legal Tag '{legal_tag_name}' already exists. Proceeding.")
    else:
        tag_response.raise_for_status() # Raise an exception for other errors

except requests.exceptions.RequestException as e:
    print(f"Error creating/checking Legal Tag: {e}")
    if e.response is not None:
        print(f"Response: {e.response.text}")
    # Stop lab if legal tag creation fails and it doesn't already exist
    if not (e.response is not None and e.response.status_code == 409):
        raise

# --- Create Placeholder Wellbore Record ---
# Define the Wellbore payload using the initial ID
wellbore_payload = {
    "kind": "osdu:wks:master-data--Wellbore:1.0.0",
    "id": wellbore_id_initial,
    "acl": {
        "owners": [f"data.default.owners@{entitlements_domain}"],
        "viewers": [f"data.default.viewers@{entitlements_domain}"]
    },
    "legal": {
        "legaltags": [legal_tag_name],
        "otherRelevantDataCountries": ["US"] # Example value
    },
    "data": {
        "FacilityName": wellbore_name,
        "FacilityTypeID": "osdu:reference-data--FacilityType:Wellbore:", # Example reference value
        "WellboreIdentity": [
            {
                "AliasName": wellbore_name,
                "AliasNameTypeID": "osdu:reference-data--AliasNameType:Name:" # Example reference value
            }
        ]
        # Add other required/optional Wellbore properties as needed
    }
}

try:
    print(f"\nAttempting to create placeholder Wellbore: {wellbore_id_initial}...")
    # Use PUT request for record creation (idempotent)
    wb_response = requests.put(
        f"{storage_endpoint}/records",
        headers=headers,
        json=[wellbore_payload] # Storage API expects a list of records
    )

    wb_response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)

    print(f"Wellbore creation/update response: {wb_response.status_code}")
    response_data = wb_response.json()
    # print(json.dumps(response_data, indent=2)) # Optional: print full response

    if wb_response.status_code in [200, 201] and response_data.get('recordIds'):
        # Get the canonical ID returned by the platform
        wellbore_id = response_data['recordIds'][0]
        print(f"Successfully created/verified Wellbore. Using ID: {wellbore_id}")
    else:
        # Fallback to using the initial ID if the response format is unexpected, but log a warning
        wellbore_id = wellbore_id_initial
        print(f"Warning: Could not extract canonical Wellbore ID from response. Proceeding with initial ID: {wellbore_id}")
        print(f"Response Body: {wb_response.text}")

except requests.exceptions.RequestException as e:
    # A 409 Conflict might occur if the wellbore with that ID already exists from a previous run
    # In this case, we can try to retrieve it to confirm and proceed.
    # However, PUT is idempotent, so a 200/201 should handle exists/created cases.
    # We'll log other errors and stop.
    print(f"Error creating/updating Wellbore: {e}")
    if e.response is not None:
        print(f"Response Status Code: {e.response.status_code}")
        print(f"Response Body: {e.response.text}")
    # Stop lab if wellbore creation fails
    raise

# --- Define path to trajectory data file ---
# This assumes a directory named 'assets' exists in the same location as the notebook
# and contains the 'trajectory.parquet' file.
# On Windows, Python typically handles forward slashes in paths correctly.
### 1.3 Google Colab Functions

# get a file from the bucket
download_blob_to_file('trajectory.parquet', 'trajectory.parquet')
trajectory_file_path = './assets/trajectory.parquet'
print(f"\nUsing trajectory data file: {trajectory_file_path}")

# Check if the file exists
if not os.path.exists(trajectory_file_path):
    print(f"Error: Trajectory data file not found at '{trajectory_file_path}'. Please ensure the file exists.")
    # Stop the lab if the data file is missing
    raise FileNotFoundError(f"Required data file not found: {trajectory_file_path}")
else:
    print("Trajectory data file found.")

print("\nLab 2 Setup Complete.")

## 1. Wellbore Trajectory Operations

### 1.1 Create WellboreTrajectory Record

This step creates the metadata record (Work Product Component - WPC) for the wellbore trajectory. This record doesn't contain the actual trajectory points yet, but describes the trajectory and links it to the Wellbore created in the setup.

UPDATES:
- "TopDepthMeasuredDepth": 0 instead of an object
- "BaseDepthMeasuredDepth": 2.0 instead of an object
- "VerticalMeasurement": {
            "EffectiveDateTime": "2021-08-17T14:13:08.174Z",
            "VerticalMeasurement": 0
        }, instead of "VerticalMeasurement": {
            "VerticalMeasurementPathID": "osdu:reference-data--VerticalMeasurementPath:Kelly Bushing:", # Example reference value
            "VerticalMeasurementTypeID": "osdu:reference-data--VerticalMeasurementType:Measured Depth:", # Example reference value
            "VerticalMeasurementSourceID": "osdu:reference-data--VerticalMeasurementSource:Estimated:" # Example reference value
        },
- ProjectedBottomHoleLocation was removed,
- AvailableTrajectoryStationProperties.UnitOfMeasuredId was replaced by StationPropertyUnitID

In [ ]:
# Define a unique ID for the trajectory record
trajectory_record_id_initial = f"{osdu_data_partition_id}:work-product-component--WellboreTrajectory:labtrajectory-{user_id}"
trajectory_record_id = None # Will be set after creation

# Trajectory WPC Payload
trajectory_payload = {
    "kind": "osdu:wks:work-product-component--WellboreTrajectory:1.1.0", # Using 1.1.0 as an example, check your platform's schema version
    "id": trajectory_record_id_initial,
    "acl": {
        "owners": [f"data.default.owners@{entitlements_domain}"],
        "viewers": [f"data.default.viewers@{entitlements_domain}"]
    },
    "legal": {
        "legaltags": [legal_tag_name],
        "otherRelevantDataCountries": ["US"]
    },
    "data": {
        "Name": f"Trajectory_{wellbore_name}",
        "Description": f"Wellbore trajectory for {wellbore_name} generated during lab session {user_id}",
        "WellboreID": f"{wellbore_id}", # Reference the created Wellbore ID
        "TopDepthMeasuredDepth": 0,
        "BaseDepthMeasuredDepth": 2.0,
        "VerticalMeasurement": {
            "EffectiveDateTime": "2021-08-17T14:13:08.174Z",
            "VerticalMeasurement": 0
        },
        # "ProjectedBottomHoleLocation": { # Added example for completeness
        #     "Latitude": {"value": 0.0, "uom": "dega"},
        #     "Longitude": {"value": 0.0, "uom": "dega"},
        #     "CoordinateReferenceSystemID": "osdu:reference-data--CoordinateReferenceSystem:WGS 84:"
        # },
        # Define available station properties based on the data to be uploaded
        "AvailableTrajectoryStationProperties": [
            {
                "TrajectoryStationPropertyTypeID": "osdu:reference-data--TrajectoryStationPropertyType:MeasuredDepth:",
                "StationPropertyUnitID": "osdu:reference-data--UnitOfMeasure:m:", # Ensure UoM matches reference data
                "Name": "Measured Depth"
            },
            {
                "TrajectoryStationPropertyTypeID": "osdu:reference-data--TrajectoryStationPropertyType:Inclination:",
                "StationPropertyUnitID": "osdu:reference-data--UnitOfMeasure:rad:", # Ensure UoM matches reference data
                "Name": "Inclination"
            },
            {
                "TrajectoryStationPropertyTypeID": "osdu:reference-data--TrajectoryStationPropertyType:AzimuthTrueNorth:",
                "StationPropertyUnitID": "osdu:reference-data--UnitOfMeasure:rad:", # Ensure UoM matches reference data
                "Name": "Azimuth"
            },
            {
                "TrajectoryStationPropertyTypeID": "osdu:reference-data--TrajectoryStationPropertyType:TVD:",
                "StationPropertyUnitID": "osdu:reference-data--UnitOfMeasure:m:", # Ensure UoM matches reference data
                "Name": "True Vertical Depth"
            }
            # Add other properties like DX, DY if present in the data
        ]
    }
}

print(f"Creating WellboreTrajectory record: {trajectory_record_id_initial}...")
try:
    response = requests.put(
        f"{storage_endpoint}/records",
        headers=headers,
        json=[trajectory_payload]
    )
    response.raise_for_status()
    print(f"WellboreTrajectory creation/update response: {response.status_code}")

    response_data = response.json()
    if response.status_code in [200, 201] and response_data.get('recordIds'):
        trajectory_record_id = response_data['recordIds'][0]
        print(f"Successfully created/verified WellboreTrajectory. Using ID: {trajectory_record_id}")
    else:
        trajectory_record_id = trajectory_record_id_initial # Fallback
        print(f"Warning: Could not extract canonical Trajectory ID. Using initial ID: {trajectory_record_id}")
        print(f"Response Body: {response.text}")

except requests.exceptions.RequestException as e:
    print(f"Error creating WellboreTrajectory record: {e}")
    if e.response is not None:
        print(f"Response Status Code: {e.response.status_code}")
        print(f"Response Body: {e.response.text}")
    # Stop if creation fails
    raise

### 1.2 Get WellboreTrajectory Metadata

This step retrieves the metadata for the WellboreTrajectory record created previously using the Wellbore DDMS API.

In [ ]:
if trajectory_record_id:
    print(f"Getting WellboreTrajectory metadata for: {trajectory_record_id}")
    try:
        # DDMS endpoint requires URL encoding for the record ID
        encoded_trajectory_id = urllib.parse.quote(trajectory_record_id)
        url = f"{wellbore_dms_endpoint}/wellboretrajectories/{encoded_trajectory_id}"
        print(f"Request URL: {url}")

        response = requests.get(url, headers=headers)
        response.raise_for_status()
        print(f"Get WellboreTrajectory metadata response status: {response.status_code}")
        print("--- WellboreTrajectory Metadata ---")
        print(json.dumps(response.json(), indent=2))
        print("-----------------------------------")
    except requests.exceptions.RequestException as e:
        print(f"Error getting WellboreTrajectory metadata: {e}")
        if e.response is not None:
            print(f"Response Body: {e.response.text}")
else:
    print("WellboreTrajectory record ID not available. Skipping Get Metadata step.")

### 1.3 Upload WellboreTrajectory Data (Parquet)

This step uploads the actual trajectory station data (MD, Inclination, Azimuth, etc.) from the local Parquet file to the OSDU platform via the Wellbore DDMS. The DDMS handles storing this data efficiently, often linking it to the metadata record.

In [ ]:
if trajectory_record_id and os.path.exists(trajectory_file_path):
    print(f"Uploading trajectory data from '{trajectory_file_path}' to record: {trajectory_record_id}")
    try:
        # Open the Parquet file in binary read mode
        with open(trajectory_file_path, "rb") as file_content:
            # Use the DDMS endpoint for uploading data, passing the record ID
            encoded_trajectory_id = urllib.parse.quote(trajectory_record_id)
            upload_url = f"{wellbore_dms_endpoint}/wellboretrajectories/{encoded_trajectory_id}/data"
            print(f"Upload URL: {upload_url}")

            # Use parquet_headers which sets 'Content-Type': 'application/x-parquet'
            response = requests.post(upload_url, headers=parquet_headers, data=file_content)
            response.raise_for_status()

            print(f"Trajectory data uploaded successfully. Response status: {response.status_code}")
            # Check if the response contains JSON data (e.g., dataset info)
            try:
                print("--- Upload Response Body ---")
                print(json.dumps(response.json(), indent=2))
                print("--------------------------")
            except json.JSONDecodeError:
                print("(Response body is not JSON)")
                print(response.text)

    except requests.exceptions.RequestException as e:
        print(f"Error uploading Parquet file: {e}")
        if e.response is not None:
            print(f"Response Status Code: {e.response.status_code}")
            print(f"Response Body: {e.response.text}")
    except FileNotFoundError:
        print(f"Error: File not found at '{trajectory_file_path}'")
    except Exception as e:
         print(f"An unexpected error occurred during upload: {e}")
else:
    print("Skipping data upload: Trajectory record ID is not available or data file is missing.")

### 1.4 Get WellboreTrajectory Data (as JSON)

This step retrieves the previously uploaded trajectory data using the Wellbore DDMS API, requesting it in JSON format.

In [ ]:
if trajectory_record_id:
    try:
        # Prepare headers to accept JSON
        get_headers = headers.copy()
        get_headers["Accept"] = "application/json"

        encoded_trajectory_id = urllib.parse.quote(trajectory_record_id)
        url = f"{wellbore_dms_endpoint}/wellboretrajectories/{encoded_trajectory_id}/data"
        print(f"Retrieving WellboreTrajectory data as JSON from: {url}")

        response = requests.get(url, headers=get_headers)
        response.raise_for_status()
        print(f"Get WellboreTrajectory data (JSON) response status: {response.status_code}")

        # Process the response
        print(f"Response Content-Type: {response.headers.get('Content-Type')}")
        if 'application/json' in response.headers.get('Content-Type', ''):
            print("--- Trajectory Data (JSON) ---")
            trajectory_data_json = response.json()
            # Print only the first few stations for brevity
            if isinstance(trajectory_data_json, list) and len(trajectory_data_json) > 5:
                 print(json.dumps(trajectory_data_json[:5], indent=2))
                 print("...")
                 print(f"(Total: {len(trajectory_data_json)} stations)")
            else:
                 print(json.dumps(trajectory_data_json, indent=2))
            print("----------------------------")
        else:
            print(f"Received unexpected content type: {response.headers.get('Content-Type')}")
            print(response.text[:1000] + "...") # Print beginning of response

    except requests.exceptions.RequestException as e:
        print(f"Error getting WellboreTrajectory data as JSON: {e}")
        if e.response is not None:
            print(f"Response body: {e.response.text}")
else:
    print("WellboreTrajectory record ID not available. Skipping Get Data (JSON) step.")

### 1.5 Get WellboreTrajectory Data (as Parquet)

This step retrieves the trajectory data again, but requests it in the original Parquet format. It then uses `pandas` and `pyarrow` (installed earlier) to read the binary Parquet data directly into a DataFrame.

In [ ]:
trajectory_data_parquet_content = None # Initialize variable to store content

if trajectory_record_id:
    try:
        # Prepare headers to accept Parquet
        get_headers = headers.copy()
        # Common MIME types for Parquet, adjust if your instance uses a different one
        get_headers["Accept"] = "application/x-parquet, application/octet-stream"

        encoded_trajectory_id = urllib.parse.quote(trajectory_record_id)
        url = f"{wellbore_dms_endpoint}/wellboretrajectories/{encoded_trajectory_id}/data"
        print(f"Retrieving WellboreTrajectory data as Parquet from: {url}")

        response = requests.get(url, headers=get_headers)
        response.raise_for_status()
        print(f"Get WellboreTrajectory data (Parquet) response status: {response.status_code}")

        # Check content type before attempting to read Parquet
        content_type = response.headers.get('Content-Type', '').lower()
        print(f"Response Content-Type: {content_type}")

        if 'parquet' in content_type or 'octet-stream' in content_type:
            print("Received Parquet or binary stream. Reading into DataFrame...")
            # Store the binary content for potential later use (like visualization)
            trajectory_data_parquet_content = response.content

            # Read the binary content directly into a Pandas DataFrame
            # BytesIO creates an in-memory binary file from the response content
            trajectory_df = pd.read_parquet(BytesIO(trajectory_data_parquet_content))

            print("--- Trajectory Data (DataFrame from Parquet) ---")
            # Display the first few rows of the DataFrame
            display(trajectory_df.head())
            print(f"(Total: {len(trajectory_df)} rows)")
            print("----------------------------------------------")

        elif 'application/json' in content_type:
            print("Received JSON instead of Parquet. Displaying JSON:")
            print(json.dumps(response.json(), indent=2))
        else:
            print(f"Received unexpected Content-Type: {content_type}. Cannot process as Parquet.")
            # Optionally save the binary content to a file for inspection
            # with open(f"trajectory_download_{user_id}.bin", "wb") as f:
            #    f.write(response.content)
            # print("Binary content saved to file.")

    except requests.exceptions.RequestException as e:
        print(f"Error getting WellboreTrajectory data as Parquet: {e}")
        if e.response is not None:
            print(f"Response body: {e.response.text}")
    except ImportError:
         print("Error: pyarrow library not found. Cannot read Parquet. Make sure it was installed.")
    except Exception as e:
        print(f"An error occurred processing the Parquet data: {e}")
else:
    print("WellboreTrajectory record ID not available. Skipping Get Data (Parquet) step.")

### 1.6 Visualize Wellbore Trajectory

This step uses the trajectory data (retrieved as Parquet in the previous step) and the `matplotlib` library to create a simple 3D plot of the wellbore path.

UPDATED:
- ['MeasuredDepth', 'Inclination', 'AzimuthTrueNorth'] replaced with ['Measured Depth', 'Inclination', 'Azimuth']
- azimuth_rad = trajectory_df['Azimuth']
- md = trajectory_df['Measured Depth']

In [ ]:
if trajectory_data_parquet_content:
    try:
        print("Visualizing trajectory data...")
        # Read the stored Parquet content into a DataFrame again
        trajectory_df = pd.read_parquet(BytesIO(trajectory_data_parquet_content))

        # --- Trajectory Calculation (Minimum Curvature Method - Simplified) ---
        # Ensure required columns exist (adjust names if different in your Parquet file)
        required_cols = ['Measured Depth', 'Inclination', 'Azimuth']
        if not all(col in trajectory_df.columns for col in required_cols):
             raise ValueError(f"Missing required columns for visualization: {required_cols}. Found: {trajectory_df.columns.tolist()}")

        # Convert Azimuth and Inclination from Radians (as per schema) to Degrees for easier understanding if needed,
        # but calculations often use radians. Assuming input is in Radians based on sample payload.
        # If your data is in degrees, convert here:
        # inclination_rad = np.radians(trajectory_df['Inclination'])
        # azimuth_rad = np.radians(trajectory_df['AzimuthTrueNorth'])
        inclination_rad = trajectory_df['Inclination']
        azimuth_rad = trajectory_df['Azimuth']
        md = trajectory_df['Measured Depth']

        # Initialize arrays for coordinates
        x = np.zeros_like(md)
        y = np.zeros_like(md)
        z = np.zeros_like(md) # TVD

        # Calculate coordinates iteratively
        for i in range(1, len(md)):
            delta_md = md[i] - md[i-1]

            # Average angles between stations
            avg_incl = (inclination_rad[i] + inclination_rad[i-1]) / 2
            avg_azim = (azimuth_rad[i] + azimuth_rad[i-1]) / 2

            # Dog Leg Severity (angle change rate)
            # More accurate methods exist, this is simplified
            # Using basic trig for displacement components
            delta_x = delta_md * np.sin(avg_incl) * np.sin(avg_azim) # Easting
            delta_y = delta_md * np.sin(avg_incl) * np.cos(avg_azim) # Northing
            delta_z = delta_md * np.cos(avg_incl) # True Vertical Depth change

            x[i] = x[i-1] + delta_x
            y[i] = y[i-1] + delta_y
            z[i] = z[i-1] + delta_z

        # If TVD is directly available in the data, prefer using it:
        if 'TVD' in trajectory_df.columns:
            print("Using 'TVD' column from data for Z-axis.")
            z = trajectory_df['TVD']
        else:
             print("Calculating TVD (Z-axis) from MD, Inclination.")

        # --- Plotting ---
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Plot the 3D trajectory
        ax.plot(x, y, z, label=f'Wellbore Trajectory ({wellbore_name})', marker='.')

        # Set labels and title
        ax.set_xlabel('East Offset (m)')
        ax.set_ylabel('North Offset (m)')
        ax.set_zlabel('True Vertical Depth (m)')
        ax.set_title('Wellbore Trajectory 3D Visualization')

        # Invert Z-axis so depth increases downwards
        ax.invert_zaxis()

        # Set aspect ratio to be equal for better spatial representation
        # ax.set_aspect('equal') # Matplotlib 3D doesn't always support 'equal' directly
        # Auto-scaling usually works well enough for demonstration
        ax.autoscale_view()

        ax.legend()
        plt.tight_layout()
        plt.show()

    except ImportError:
        print("Error: Matplotlib or NumPy not found. Cannot visualize. Make sure they were installed.")
    except KeyError as e:
        print(f"Error visualizing: Missing column in trajectory data: {e}")
    except Exception as e:
        print(f"An error occurred during visualization: {e}")
else:
    print("Trajectory data (Parquet content) not available. Skipping visualization.")

## Cleanup for Lab 2

This section deletes the resources created during Lab 2 to keep the OSDU instance clean. It attempts to delete:
*   The WellboreTrajectory record.
*   The placeholder Wellbore record.
*   The Legal Tag.

**Note:** Deleting DDMS data associated with a record (like the trajectory points uploaded in step 1.3) typically happens automatically when the metadata record (WellboreTrajectory WPC) is deleted via the Storage API, but this behavior might depend on the specific DDMS implementation and platform configuration.

In [ ]:
print("--- Starting Lab 2 Cleanup ---")

# 1. Delete the WellboreTrajectory record
if 'trajectory_record_id' in locals() and trajectory_record_id:
    print(f"\nAttempting to delete WellboreTrajectory record: {trajectory_record_id}")
    try:
        delete_url = f"{storage_endpoint}/records/{urllib.parse.quote(trajectory_record_id)}"
        # Storage API V2 delete returns 204 No Content on success
        response = requests.delete(delete_url, headers=headers)
        if response.status_code == 204:
            print(f"Successfully deleted WellboreTrajectory record: {trajectory_record_id}")
        elif response.status_code == 404:
             print(f"WellboreTrajectory record {trajectory_record_id} already deleted or not found.")
        else:
            response.raise_for_status() # Raise error for other statuses
    except requests.exceptions.RequestException as e:
        print(f"Error deleting WellboreTrajectory record {trajectory_record_id}: {e}")
        if e.response is not None:
            print(f"Response: {e.response.text}")
else:
    print("\nWellboreTrajectory record ID not available. Skipping deletion.")

# 2. Delete the placeholder Wellbore record
if 'wellbore_id' in locals() and wellbore_id:
    print(f"\nAttempting to delete Wellbore record: {wellbore_id}")
    try:
        delete_url = f"{storage_endpoint}/records/{urllib.parse.quote(wellbore_id)}"
        response = requests.delete(delete_url, headers=headers)
        if response.status_code == 204:
            print(f"Successfully deleted Wellbore record: {wellbore_id}")
        elif response.status_code == 404:
             print(f"Wellbore record {wellbore_id} already deleted or not found.")
        else:
            response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error deleting Wellbore record {wellbore_id}: {e}")
        # Check if it failed because it's referenced by other records (like the trajectory if deletion failed)
        if e.response is not None:
            print(f"Response: {e.response.text}")
else:
    print("\nWellbore record ID not available. Skipping deletion.")

# 3. Delete the Legal Tag
if 'legal_tag_name' in locals() and legal_tag_name:
    print(f"\nAttempting to delete Legal Tag: {legal_tag_name}")
    try:
        delete_url = f"{legal_endpoint}/legaltags/{urllib.parse.quote(legal_tag_name)}"
        # Legal API delete also typically returns 204 No Content
        response = requests.delete(delete_url, headers=headers)
        if response.status_code == 204:
            print(f"Successfully deleted Legal Tag: {legal_tag_name}")
        elif response.status_code == 404:
             print(f"Legal Tag {legal_tag_name} already deleted or not found.")
        else:
            response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error deleting Legal Tag {legal_tag_name}: {e}")
        # Check if it failed because it's still in use by records
        if e.response is not None:
            print(f"Response: {e.response.text}")
else:
    print("\nLegal Tag name not available. Skipping deletion.")

print("\n--- Lab 2 Cleanup Complete ---")

## Challenge (Optional)

**Task:** Modify the lab to work with Well Logs instead of Wellbore Trajectories.

1.  **Adapt Setup:** Keep the Legal Tag and Wellbore creation.
2.  **Create WellLog Record:** Create a `work-product-component--WellLog` record instead of a `WellboreTrajectory`. Populate its metadata appropriately (e.g., `Name`, `WellboreID`, `Curves`). You'll need to define which curves the log contains (e.g., GR, CALI, DEPTH). Find a sample WellLog Parquet/CSV/LAS file or create a small dummy one.
3.  **Upload Log Data:** Use the Wellbore DDMS endpoint for WellLogs (`/welllogs/{record_id}/data`) to upload your sample log data (e.g., in Parquet format).
4.  **Retrieve Log Data:** Use the corresponding GET endpoint (`/welllogs/{record_id}/data`) to retrieve the log data, potentially requesting specific curves or a depth range.
5.  **Visualize:** Plot one or two curves (e.g., GR vs. DEPTH) using Matplotlib.
6.  **Cleanup:** Ensure the WellLog record is deleted in the cleanup section.

# Lab 3: Reservoir DMS Operations

This tutorial focuses on understanding the components of the RDDMS and how to use the ETP and REST API.

In the following diagram representing a typical OSDU deployment, you will notice that:
- The ETP server and REST API service will be running alongside OSDU services within a Kubernetes cluster.
- The RDDMS services interact with other OSDU services, such as entitlements, records, and more.
- The ETP server can connect to multiple OSDU partitions.
- The ETP and REST endpoints are exposed via secured protocols (HTTPS and WSS).
- Authentication and authorization are enabled.

The API documentation should be available on the [Swagger page](https://preship.gcp.gnrg-osdu.projects.epam.com/api/reservoir-ddms/v2/).


## Setup for Lab 3

The following sections at the top of this workbook must be executed prior starting this lab.

- [OSDU Hands-on Lab Environment Setup](OSDU_Bootcamp_Labs.ipynb#osdu-hands-on-lab-environment-setup)
  - [1 Initial Setup](OSDU_Bootcamp_Labs.ipynb#1-initial-setup)
    - [1.2 Configure Connection and Authentication](OSDU_Bootcamp_Labs.ipynb#12-configure-connection-and-authentication)

### Auth in Google Cloud

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import auth
    auth.authenticate_user()

    osdu_endpoint = "https://osdu-bootcamp.com"
    osdu_data_partition_id= "osdu"
    osdu_group_domain = "group"


    def get_token():
        output = !gcloud auth print-access-token
        token = output[0]
        return token
else:
    # clear previous token and set up new environment
    _token_context = {
        "token": None,
        "expires_at": 0
    }

    # Configure and authenticate to OSDU instance with RDDMS
    dotenv_file = ".env.osdu-bootcamp"
    initialize_from_env_file(dotenv_file)

In [ ]:
# Generate Personal Identifier
if( 'user_id' not in locals() or not user_id):
    import random
    user_id = f"{random.randint(1000, 9999)}"
    print(f"Generated user_id for this session: {user_id}")

# Functions to display a banner message
def green_banner(msg, detail_msg=None):
    if detail_msg:
        text = f"{msg}<br><span style='font-size: 12pt;'>{detail_msg}</span>"
    else:
        text = msg
    return display_banner(text, color='green')

def red_banner(msg, detail_msg=None):
    if detail_msg:
        text = f"{msg}<br><span style='font-size: 12pt;'>{detail_msg}</span>"
    else:
        text = msg
    return display_banner(text, color='red')

# Ensure osdu_data_partition_id is defined
if 'osdu_data_partition_id' not in locals() or not osdu_data_partition_id:
    red_banner("osdu_data_partition_id is not defined.<br>Please run \"1.2 Configure Connection and Authentication\" before running this code.")
    raise SystemExit("osdu_data_partition_id is not defined. Please set osdu_data_partition_id before running this code.")

# Ensure osdu_endpoint is defined
if 'osdu_endpoint' not in locals() or not osdu_endpoint:
    red_banner("osdu_endpoint is not defined.<br>Please run \"1.2 Configure Connection and Authentication\" before running this code.")
    raise SystemExit("osdu_endpoint is not defined. Please set osdu_endpoint before running this code.")

green_banner("You are now pointing to a new OSDU instance.", f"{osdu_endpoint}")
print(osdu_endpoint)

In [ ]:
# Installing Libs needed
%pip install plotly numpy pandas requests --quiet

# importing python modules used in this notebook
import requests
import json
import pandas as pd
import numpy as np
from IPython.display import display
import plotly.graph_objects as go


# Define the endpoints that will be used in the lab
entitlements_endpoint = f"{osdu_endpoint}/api/entitlements/v2"
legal_endpoint = f"{osdu_endpoint}/api/legal/v1"
search_endpoint = f"{osdu_endpoint}/api/search/v2"

# Define  the group domain for the entitlements
entitlements_domain = f"{osdu_data_partition_id}.{osdu_group_domain}"

# function to display a pandas dataframe in a pretty format
def pretty_print_panda_frame(df, title=None, transpose=False, show_index=False, show_header=True):
    styled = (df.transpose() if transpose else df).style.set_properties(**{'text-align': 'left'})
    if title: display(HTML(f"<h3 style='margin-bottom: 0px'>{title}</h3>"))
    if not show_index: styled = styled.hide(axis='index')
    if not show_header: styled = styled.set_table_styles([{'selector': 'thead', 'props': [('display', 'none')]}])
    else: styled = styled.set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])
    display(styled)


# preparing standard REST headers
def get_rddms_header():
    return {
        "Authorization": f"Bearer {get_token()}",
        "data-partition-id": osdu_data_partition_id,
        "Content-Type": "application/json"
    }

def find_all_group_memberships():
    try:
        # Send a GET request to the entitlements endpoint to retrieve all groups
        get_groups_response = requests.get(
            f"{entitlements_endpoint}/groups",
            headers=get_rddms_header()
        )

        get_groups_response.raise_for_status()
        if get_groups_response.status_code == 200:
            get_all_groups_response = get_groups_response.json()
            group_names = [group.get('name') for group in get_all_groups_response.get('groups', [])]

            return group_names
        else:
            return []
    except requests.exceptions.RequestException as e:
        return []

# Check if the group name is in the list of all group names
def check_group_membership(group_name, all_group_names):
    if group_name in all_group_names:
        return True, f"User is member of '{group_name}'"
    else:
        return False, f"User is not member of '{group_name}'"

# Check if the legal tag exists
def check_legal_tag_exists(tag_name):
    try:
        legal_tag_check_response = requests.get(
            f"{legal_endpoint}/legaltags/{tag_name}",
            headers=get_rddms_header()
        )
        legal_tag_check_response.raise_for_status()
        if legal_tag_check_response.status_code == 200:
            return True, f"Legal tag exists: {tag_name}"
        else:
            return False, f"Legal tag does not exist: {tag_name}"
    except requests.exceptions.RequestException as e:
        return False, f"Legal tag does not exist: {tag_name}"

def create_legal_tag(tag_name, tag_description):
    legal_tag_payload = {
        "name": tag_name,
        "description": tag_description,
        "properties": {
            "contractId": f"CID-{user_id}-A1",
            "countryOfOrigin": ["US"], # Must be an ISO 3166-1 alpha-2 country code
            "dataType": "Public Domain Data", # Value must exist in reference data 'LegalTagPropertyType' for 'dataType'
            "expirationDate": "2099-12-31", # Format YYYY-MM-DD
            "exportClassification": "EAR99", # Value must exist in reference data 'LegalTagPropertyType' for 'exportClassification'
            "originator": f"LabUser-{user_id}",
            "personalData": "No Personal Data", # Value must exist in reference data 'LegalTagPropertyType' for 'personalData'
            "securityClassification": "Public" # Value must exist in reference data 'LegalTagPropertyType' for 'securityClassification'
        }
    }
    try:
        tag_response = requests.post(
            f"{legal_endpoint}/legaltags",
            headers=get_rddms_header(),
            data=json.dumps(legal_tag_payload)
        )
        if tag_response.status_code == 201:
            created_tag_name = tag_response.json().get("name")
            msg = f"Successfully created legal tag: {created_tag_name}"
        elif tag_response.status_code == 409:
            created_tag_name = tag_name
            msg = f"Legal tag '{tag_name}' already exists. Proceeding."
        else:
            created_tag_name = None
            msg = f"Error: {tag_response.status_code} - {tag_response.text}"
    except requests.exceptions.RequestException as e:
        created_tag_name = None
        if e.response is not None:
            msg = f"Response: {e.response.text}"
        else:
            msg = f"Error: {e}"
    return created_tag_name, msg

# Create a Legal Tag
user_tag_name = f'{osdu_data_partition_id}-lab-rddms-tag-{user_id}'
user_tag_description = f"Lab tag created by user {user_id}"
created_tag_name, msg = create_legal_tag(user_tag_name, user_tag_description)
if not created_tag_name:
    red_banner(f"Failed to create legal tag '{user_tag_name}'.", msg)
    raise SystemExit("Failed to create legal tag.")

green_banner("Lab 3 - Reservoir DDMS is setup and ready to go.")

## 1. Reading Data from RDDMS with REST

In this section we will:
1. Explore all dataspace(s) available on the RDDMS.
2. Select the `demo/Volve` dataspace.
3. Find information about horizon interpretations present in the dataspace.
4. Retrieve detailed information about the first horizon.
5. Use NumPy and Plotly to display the depth values of that horizon.


### 1.1 Locating the RDDMS

We define a variable `RDDMS_REST_API` that points to the REST API endpoint of the RDDMS.

In [ ]:
RDDMS_REST_API = f"{osdu_endpoint}/api/reservoir-ddms/v2"
RDDMS_ETP_API = f"{osdu_endpoint.replace('http', 'ws')}/api/reservoir-ddms-etp/v2/"

try:
    response = requests.get(f"{RDDMS_REST_API}/")
    response.raise_for_status()
    green_banner("RDDMS REST API is responding.")
except requests.exceptions.RequestException as e:
    red_banner("Error connecting to RDDMS REST API.")
    print(f"Error connecting to RDDMS REST API: {e}")
    if e.response is not None:
        print(f"Response Body: {e.response.text}")

print(f"RDDMS REST API at: {RDDMS_REST_API}/")
print(f"RDDMS ETP API at: {RDDMS_ETP_API}")


### 1.2 Listing All available Dataspaces

Retrieve a list of available dataspace(s) in the RDDMS instance.

You must be a member of the `service.reservoir-dms.viewers`

In [ ]:
# Pre-requisite check
all_groups = find_all_group_memberships()
rddms_service_viewers = 'service.reservoir-dms.viewers'
ok, msg = check_group_membership(rddms_service_viewers, all_groups)
if not ok:
    detail_text = f"Please contact your administrator to add you to the group."
    red_banner(msg, detail_text)

# use the GET '/dataspaces' API to list all dataspaces
all_dataspaces_response = requests.get(f"{RDDMS_REST_API}/dataspaces", headers=get_rddms_header())
# print(json.dumps(all_dataspaces_response.json(),indent=4))
if all_dataspaces_response.status_code != 200:
    detail_text = f"Error: {all_dataspaces_response.status_code} - {all_dataspaces_response.text}"
    red_banner(f"Failed to retrieve dataspaces.", detail_text)

# transforming the json array into a pandas dataframe.
data_list = all_dataspaces_response.json()

if all_dataspaces_response.status_code == 200 :
    # Step 1: Extract relevant fields from data_list
    data_list_processed = [
        {"Path": item["path"], "URI": item["uri"], "Created": item["storeCreated"], "Updated": item["storeLastWrite"], "Locked" : item["customData"]["locked"]}
        for item in data_list
    ]

    # Step 2: Create a DataFrame with the extracted fields
    df = pd.DataFrame(data_list_processed )

    # Print Dataspaces as a table
    pretty_print_panda_frame(df, title=f"All Dataspaces (Total: {len(df)})")

### 1.3 Searching and Selecting `demo/Volve`

Search for the `demo/Volve` dataspace and select it for further operations.

In [ ]:
RDDMS_DATASPACE = 'demo/Volve'
try:
    SelectedDataspace = next(item for item in all_dataspaces_response.json() if item["path"] == RDDMS_DATASPACE)
    dataspace_name = urllib.parse.quote(SelectedDataspace['path'], safe="")
    green_banner(f"DATASPACE '{SelectedDataspace['path']}' SELECTED SUCCESSFULLY")

except:
    red_banner(f"DATASPACE '{RDDMS_DATASPACE}' NOT FOUND")

### 1.4 Printing Data Type Statistics for the Dataspace

Display statistics on the data types contained within the `demo/Volve` dataspace.

In [ ]:
# use the GET 'dataspaces/{dataspace_name}/resources' API to list all resources in the dataspace
resources_response = requests.get(f'{RDDMS_REST_API}/dataspaces/{dataspace_name}/resources', headers=get_rddms_header())

if resources_response.status_code != 200:
    red_banner(f"Failed to read data from the dataspace '{RDDMS_DATASPACE}'")

#print(json.dumps(response.json(),indent=4))

# print all the datatypes in the dataspace
response_flat = pd.json_normalize(resources_response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
pretty_print_panda_frame(response_df, title="All datatypes in the dataspace")

### 1.5 Listing All Seismic Horizons

Retrieve and list all objects of type `resqml20.obj_Grid2dRepresentation`, which is used to represent interpreted Seismic horizons.

In [ ]:
# use the GET 'dataspaces/{dataspace_name}/resources/{datatype}' API to list all object of the given datatype
resqml_datatype = 'resqml20.obj_Grid2dRepresentation'
horizons_response = requests.get(
    f'{RDDMS_REST_API}/dataspaces/{dataspace_name}/resources/{resqml_datatype}',
    headers=get_rddms_header()
)
if horizons_response.status_code != 200:
    red_banner(f"Failed to retrieve Interpreted Horizons")

# Extract relevant information from the JSON response
horizon_data = horizons_response.json()
horizon_table = [
    {
        "Horizon Name": item.get("name", "N/A"),
        "Creator Name": item.get("customData", {}).get("creator", "N/A"),
        "Created Date": item.get("customData", {}).get("created", "N/A").split("T")[0],  # Extract only the date part (YYYY-MM-DD)
        "URI": item.get("uri", "N/A")
    }
    for item in horizon_data
]

# Convert to a pandas DataFrame for better visualization
horizon_df = pd.DataFrame(horizon_table)
pretty_print_panda_frame(horizon_df, title="All Interpreted Horizons in the dataspace")

# Raw JSon response
# print(json.dumps(response.json(),indent=4))


### 1.6 Selecting the First Horizon Interpretation

Choose the first horizon interpretation from the available data in the dataspace.

In [ ]:
### Selecting the First Horizon Interpretation
# Choose the first horizon interpretation from the available data in the dataspace.
################################################

SelectedSurface =  1 # first object in dataspace object list
horizon_uuid = urllib.parse.quote(horizons_response.json()[SelectedSurface]['uri'].split('(')[-1].replace(')',''))

print('First Horizon uuid:', horizon_uuid)
#print('Detail record of the first horizon')
#print(json.dumps(horizons_response.json()[SelectedSurface],indent=4))

def process_horizon_metadata(RDDMS_REST_API, dataspace_name, resqml_datatype, horizon_uuid, headers):
    params = {
        '$format': 'json',
        'arrayMetadata': 'false',
        'arrayValues': 'false',
        'referencedContent': 'true',
    }

    # Retrieve metadata for the first horizon
    first_horizon_response = requests.get(
        f'{RDDMS_REST_API}/dataspaces/{dataspace_name}/resources/{resqml_datatype}/{horizon_uuid}',
        params=params,
        headers=headers
    )
    if first_horizon_response.status_code != 200:
        red_banner(f"Failed to retrieve metadata for the first horizon")
        return None, None

    def extract_grid_geometry(json_data):
        try:
            grid_patch = json_data[0]["Grid2dPatch"]
            # print(f"Grid2dPatch: {json.dumps(grid_patch, indent=4)}")
            if grid_patch["Geometry"]["Points"]["$type"] == "resqml20.Point3dLatticeArray":
                geometry = grid_patch["Geometry"]["Points"]
            elif grid_patch["Geometry"]["Points"]["$type"] == "resqml20.Point3dZValueArray":
                geometry = grid_patch["Geometry"]["Points"]["SupportingGeometry"]["SupportingRepresentation"]["_data"]["Grid2dPatch"]["Geometry"]["Points"]
            else :
                return None

            origin = [geometry["Origin"]["Coordinate1"], geometry["Origin"]["Coordinate2"]]
            spacing = [
                offset["Spacing"]["Value"] for offset in geometry.get("Offset", [])
            ]
            vectors =[
                [offset["Offset"]["Coordinate1"], offset["Offset"]["Coordinate2"]] for offset in geometry.get("Offset", [])
            ]
            sizes = [grid_patch["FastestAxisCount"], grid_patch["SlowestAxisCount"]]

            u_vector, v_vector = np.array(vectors) * spacing
            # Check if the two vectors are orthogonal
            dot_product = np.dot(vectors[0], vectors[1])
            is_orthogonal = np.isclose(dot_product, 0, atol=1e-6)
            print(f"Are the vectors orthogonal? {'Yes' if is_orthogonal else 'No'}")

            return {
                "Origin": origin,
                "Vector": [u_vector, v_vector],
                "Size": sizes
            }
        except (KeyError, IndexError, TypeError) as e:
            print(f"Error extracting grid origin: {e}")
            return None

    # Extract grid geometry details
    horizon_geometry = extract_grid_geometry(first_horizon_response.json())
    if not horizon_geometry:
        red_banner(f"Failed to extract grid geometry details")
        return None, None

    # Retrieve data arrays for the horizon
    arrays_response = requests.get(
        f'{RDDMS_REST_API}/dataspaces/{dataspace_name}/resources/{resqml_datatype}/{horizon_uuid}/arrays',
        params=params,
        headers=headers
    )
    if arrays_response.status_code != 200:
        red_banner(f"Failed to retrieve data arrays for the horizon")
        return None, None

    return horizon_geometry, arrays_response

horizon_geometry, arrays_response = process_horizon_metadata(RDDMS_REST_API, dataspace_name, resqml_datatype, horizon_uuid, headers = get_rddms_header())

# Create a DataFrame
df = pd.DataFrame(horizon_geometry)
pretty_print_panda_frame(df, title="Horizon Geometry", transpose=True, show_index=True, show_header=False)


### 1.7 Reading the Depth Array values

Extract the depth array data from the RDDMS for the selected horizon.

In [ ]:
# This code processes the depth array for the selected horizon and calculates statistics.
# It retrieves the depth array data, reshapes it, and calculates statistics such as min, max, mean, and standard deviation.
# The results are then displayed in a pandas DataFrame for better visualization.

def process_horizon_depth_array(RDDMS_REST_API, dataspace_name, resqml_datatype, horizon_uuid, arrays_response, headers):
    # Getting the URL encoded of the path to the array.
    array_uuid_url = urllib.parse.quote(arrays_response.json()[0]['uid']['pathInResource'], safe="")

    params = {
        'format': 'json'
    }

    # use the REST API to return the depth array as a JSON array
    array_response = requests.get(
        f'{RDDMS_REST_API}/dataspaces/{dataspace_name}/resources/{resqml_datatype}/{horizon_uuid}/arrays/{array_uuid_url}',
        params=params,
        headers=headers
    )

    if array_response.status_code != 200:
        red_banner("Failed to retrieve the depth array")
        return None
    return array_response

if not arrays_response.json():
    red_banner("No arrays found for the selected horizon")
else:
    array_response = process_horizon_depth_array(RDDMS_REST_API, dataspace_name, resqml_datatype, horizon_uuid, arrays_response, headers=get_rddms_header())
    if array_response.status_code != 200:
        red_banner("Failed to retrieve the depth array")

    # extract the data array size
    print('Data length:',len(array_response.json()['data']['data']))
    print('Dimensions:',array_response.json()['data']['dimensions'])
    if len(array_response.json()['data']['data']) != (int(array_response.json()['data']['dimensions'][0]) * int(array_response.json()['data']['dimensions'][1])):
        red_banner("Data length does not match the expected dimensions!")

    z_data = np.array(array_response.json()['data']['data'], dtype=np.float32)
    # Reshape z_data to match the dimensions and create a DataFrame
    z_data_reshaped = z_data.reshape(array_response.json()['data']['dimensions'])
    # Calculate statistics for the reshaped data
    z_stats = {
        "Min": [np.min(z_data_reshaped)],
        "Max": [np.max(z_data_reshaped)],
        "Mean": [np.mean(z_data_reshaped)],
        "Std Dev": [np.std(z_data_reshaped)]
    }

    # Create a DataFrame with the statistics
    z_df = pd.DataFrame(z_stats)

    # Display the DataFrame with pretty formatting
    pretty_print_panda_frame(z_df, title="Depth Array Statistics", transpose=True, show_index=True, show_header=False)


### 1.8 Using Plotly to Display the Depth Array

Visualize the depth array using Plotly to have a 3D representation of the horizon.

In [ ]:
# Returns the X, Y and Z arrays for a given horizon URI
def get_horizon_arrays(RDDMS_REST_API, dataspace_name, horizon_uuid, headers):
    # Get the metadata for the horizon
    geometry, arrays_response = process_horizon_metadata(RDDMS_REST_API, dataspace_name, resqml_datatype, horizon_uuid, headers)

    # Build the grid geometry as X and Y 2D arrays
    sizes = geometry['Size']
    origin = geometry['Origin']
    vectors = geometry['Vector']
    u_vector, v_vector = np.array(vectors).T
    i, j = np.meshgrid(np.arange(sizes[1]), np.arange(sizes[0]), indexing='ij')
    X = origin[0] + i * u_vector[0] + j * v_vector[0]
    Y = origin[1] + i * u_vector[1] + j * v_vector[1]

    if not arrays_response.json():
        Z = np.zeros((sizes[1], sizes[0]))
    else:
        array_response = process_horizon_depth_array(RDDMS_REST_API, dataspace_name, resqml_datatype, horizon_uuid, arrays_response, headers)
        z_data = np.array(array_response.json()['data']['data'], dtype=np.float32)
        Z = np.reshape(z_data, (int(array_response.json()['data']['dimensions'][0]), int(array_response.json()['data']['dimensions'][1])))

    return X, Y, Z


horizon_uuid1 = urllib.parse.quote(horizons_response.json()[1]['uri'].split('(')[-1].replace(')',''))

X1, Y1, Z1 = get_horizon_arrays(RDDMS_REST_API, dataspace_name, horizon_uuid1, headers=get_rddms_header());

# Get the minimum and maximum depth values
z_min, z_max = Z1.min(), Z1.max()

# Create an interactive 3D surface plot using Plotly
fig = go.Figure(data=[
    go.Surface(z=Z1, x=X1, y=Y1, colorscale='Viridis', cmin=z_min, cmax=z_max),
    ])

# Set Z range limits and camera view
fig.update_layout(
    title='3D Plot of Seismic Horizon',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Depth (m)',
        zaxis=dict(range=[3200, 2000]),
        aspectmode='manual',
    ),
    width=600,
    height=600,
)

# Show the interactive plot
fig.show()

## 2. Reading several horizons (2d grid) with FETPAPI
FETPAPI is a multilanguages (C++, Java, C# and obviously Python) ETP1.2 client library. It is used with FESAPI which is a multilanguages (C++, Java, C# and obviously Python) RESQML2 client library

### 2.1 Setup for FETAPI

In [ ]:
# Installing Libs needed
%pip install fetpapi --quiet

# importing python modules used in this notebook
import fesapi
import fetpapi
import uuid
import re
import random

### 2.2 Open an ETP websocket (persisted) session


In [ ]:
initialization_params = fetpapi.InitializationParameters(str(uuid.uuid4()), RDDMS_ETP_API)

additionalHeaderField = fetpapi.MapStringString()
additionalHeaderField["data-partition-id"] = osdu_data_partition_id
initialization_params.setAdditionalHandshakeHeaderFields(additionalHeaderField)

client_session = fetpapi.createClientSession(initialization_params, f"Bearer {get_token()}")

# Run the session in parallel not to block the main thread
def start_etp_server(client_session):
    client_session.run()

from threading import Thread
t = Thread(target=start_etp_server, args=(client_session,), daemon=True)
t.start()

# Wait for the connection to be effective
from time import sleep, perf_counter
start_time = perf_counter()
while client_session.isEtpSessionClosed() and perf_counter() - start_time < 5:
    sleep(0.25)
if client_session.isEtpSessionClosed():
    red_banner("The ETP session could not be established in 5 seconds.")
    if not RDDMS_ETP_API.endswith("/"):
        print("You may try adding an ending slash to OSDU RDDMS URL.")
else:
    green_banner(f"Now connected to OSDU RDDMS at {RDDMS_ETP_API}")

### 2.3 Create a FESAPI DataObject Repository to store in memory the OSDU horizon
We also set it up to deal with the ETP DataArray subprotocol

In [ ]:
repo = fesapi.DataObjectRepository()
# Create a specialized HdfProxy to deal with ETP DataArray subprotocol
hdf_proxy_factory = fetpapi.FesapiHdfProxyFactory(client_session)
repo.setHdfProxyFactory(hdf_proxy_factory)
green_banner("FESAPI DataObjectRepository created")

### 2.4 Load all dataobjects from OSDU RDDMS relevant dataspace into the DataObjectRepository
Data Arrays are not loaded yet

In [ ]:
# ETP uri arrays should be given from the catalog search instead
etp_uris = [entry['URI'] for entry in horizon_table]

# Identifity the dataspace where the data are stored
dataspace_uri = re.search(r"(eml:///dataspace\('[^']+'\))", etp_uris[0]).group(1)
if not all(entry['URI'].startswith(dataspace_uri) for entry in horizon_table):
    red_banner(f"All etp uris must be in the same dataspace.")
else :
    # Discover all resources from the dataspace
    etp_context = fetpapi.ContextInfo()
    etp_context.uri = dataspace_uri
    etp_context.depth = 1
    etp_context.navigableEdges = fetpapi.RelationshipKind_Both
    etp_context.includeSecondaryTargets = False
    etp_context.includeSecondarySources = False
    all_resources = client_session.getResources(etp_context, fetpapi.ContextScopeKind__self)
    if all_resources.empty() :
        red_banner(f"There is no resource in dataspace {dataspace_uri}")
    else :
        green_banner(f"There are {str(len(all_resources))} resources in dataspace {dataspace_uri}")

        # Get all data objects from the resources
        uriMap = fetpapi.MapStringString();
        for index, resource in enumerate(all_resources):
            uriMap[str(index)] = resource.uri
        all_resources = client_session.getDataObjects(uriMap);
        for dataObject in all_resources.values():
            repo.addOrReplaceGsoapProxy(dataObject.data, fetpapi.getDataObjectType(dataObject.resource.uri), fetpapi.getDataspaceUri(dataObject.resource.uri))
        green_banner("Dataspace loaded in memory.")

### 2.5 Reading the Depth Array values

Extract the depth array data from the RDDMS for the selected horizons.

In [ ]:
z_data = {}
# Loop through the ETP URIs and process each horizon
for etp_uri in etp_uris:
    horizon_uuid = urllib.parse.quote(etp_uri.split('(')[-1].replace(')',''))
    horizon = repo.getDataObjectByUuid(horizon_uuid)

    grid2dNodeCount = horizon.getXyzPointCountOfAllPatches()
    if isinstance(horizon, fesapi.Resqml2_Grid2dRepresentation):
        z_values = fesapi.DoubleArray(grid2dNodeCount)
        horizon.getZValues(z_values)

        z_data[horizon] = np.empty(grid2dNodeCount, dtype=np.float64)
        for i in range(grid2dNodeCount):
            z_data[horizon][i] = z_values.getitem(i)

        # Calculate statistics for the reshaped data
        z_stats = {
            "Min": [np.nanmin(z_data[horizon])],
            "Max": [np.nanmax(z_data[horizon])],
            "Mean": [np.nanmean(z_data[horizon])],
            "Std Dev": [np.nanstd(z_data[horizon])]
        }

        # Create a DataFrame with the statistics
        z_df = pd.DataFrame(z_stats)

        # Display the DataFrame with pretty formatting
        pretty_print_panda_frame(z_df, title=f"Depth Array Statistics of {horizon.getTitle()}", transpose=True, show_index=True, show_header=False)
    else :
        red_banner(f"The UUID {horizon_uuid} does not correspond to a horizon 2d grid.")

### 2.6 Reading the first 5 faults of the dataspace

In [ ]:
fault_x_data = {}
fault_y_data = {}
fault_z_data = {}

all_faults_count = repo.getFaultPolylineSetRepresentationCount()
for fault_index in range(min(5, all_faults_count)):
    fault = repo.getFaultPolylineSetRepresentation(fault_index)

    fault_x_data[fault] = []
    fault_y_data[fault] = []
    fault_z_data[fault] = []

    node_count = fault.getXyzPointCountOfAllPatches()
    xyz_values = fesapi.DoubleArray(node_count * 3)
    fault.getXyzPointsOfAllPatches(xyz_values)

    # Count of polylines in the fault
    polyline_count = fault.getPolylineCountOfAllPatches()
    # Count of nodes per polyline in the fault
    node_count_per_polyline = fesapi.UInt32Array(polyline_count)
    fault.getNodeCountPerPolylineOfAllPatches(node_count_per_polyline.cast())

    node_index = 0
    for polyline_index in range(polyline_count):
        # Get the number of nodes in the current polyline
        polyline_node_count = node_count_per_polyline.getitem(polyline_index)

        fault_x_data[fault].append(np.empty(polyline_node_count, dtype=np.float64))
        fault_y_data[fault].append(np.empty(polyline_node_count, dtype=np.float64))
        fault_z_data[fault].append(np.empty(polyline_node_count, dtype=np.float64))

        for polyline_node_index in range(polyline_node_count):
            fault_x_data[fault][polyline_index][polyline_node_index] = xyz_values.getitem(node_index*3)
            fault_y_data[fault][polyline_index][polyline_node_index] = xyz_values.getitem(node_index*3+1)
            fault_z_data[fault][polyline_index][polyline_node_index] = xyz_values.getitem(node_index*3+2)
            node_index += 1

    green_banner(f"Fault {fault.getTitle()} has {polyline_count} polylines")

green_banner(f"All faults have been loaded in memory.")

### 2.7 Close the ETP websocket session

In [ ]:
client_session.close()
green_banner("ETP session is now closed")

### 2.8 Using Plotly to Display the Depth Arrays

Visualize the depth arrays using Plotly to have a 3D representation of the horizons.

In [ ]:
surfaces = []

global_z_min = float('inf')
global_z_max = float('-inf')

# Horizons
for horizon, z_array in z_data.items():
    # Get the minimum and maximum depth values
    Z = z_array.reshape(horizon.getNodeCountAlongJAxis(), horizon.getNodeCountAlongIAxis()).transpose()
    z_min, z_max = np.nanmin(Z), np.nanmax(Z)
    if z_min == z_max and z_min == 0:
        print(f"Skipping horizon {horizon.getTitle()} as all values are the same (zero).")
        continue  # Skip if all values are the same

    global_z_min = min(global_z_min, z_min)
    global_z_max = max(global_z_max, z_max)

show_scale = True
for horizon, z_array in z_data.items():
    i, j = np.meshgrid(np.arange(horizon.getNodeCountAlongIAxis()), np.arange(horizon.getNodeCountAlongJAxis()), indexing='ij')
    X = horizon.getXOrigin() + i * horizon.getXIOffset() + j * horizon.getXJOffset()
    Y = horizon.getYOrigin() + i * horizon.getYIOffset() + j * horizon.getYJOffset()
    Z = z_array.reshape(horizon.getNodeCountAlongJAxis(), horizon.getNodeCountAlongIAxis()).transpose()

    surfaces.append(go.Surface(z=Z, x=X, y=Y, colorscale='Viridis', cmin=global_z_min, cmax=global_z_max, showscale=show_scale, colorbar=dict(title="Depth")))
    show_scale = False  # Show color scale only with the first surface

# Faults
for fault, polyline_set in fault_z_data.items():
    random_color = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'
    for polyline_index, z_array in enumerate(polyline_set):
        # Get the minimum and maximum depth values
        z_min, z_max = np.nanmin(z_array), np.nanmax(z_array)

        global_z_min = min(global_z_min, z_min)
        global_z_max = max(global_z_max, z_max)

        surfaces.append(go.Scatter3d(
            z=z_array,
            x=fault_x_data[fault][polyline_index],
            y=fault_y_data[fault][polyline_index],
            mode='lines',
            line=dict(color=random_color, width=2),
            showlegend=False
        ))

# Create an interactive 3D surface plot using Plotly
fig = go.Figure(data=surfaces)

# Set Z range limits and camera view
fig.update_layout(
    title='3D Plot of Seismic Horizons and Faults',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Depth (m)',
        zaxis=dict(range=[global_z_max, global_z_min]),  # Flip the Z-axis by reversing the range
        aspectmode='manual',
    ),
    width=1200,
    height=600
)

# Show the interactive plot
fig.show()

## 3. Managing Dataspace
Note the user must be  in the `service.reservoir-dms.owners` service group in order to create or delete dataspaces

### 3.1 Checking Prerequisite

In [ ]:
# Define the new dataspace payload
rddms_service_owners = f"service.reservoir-dms.owners"
rddms_service_viewers = f"service.reservoir-dms.viewers"
data_owners = f"data.default.owners"
data_viewers = f"data.default.viewers"
group_names = find_all_group_memberships()
ok1, msg1 = check_group_membership(rddms_service_owners, group_names)
ok2, msg2 = check_group_membership(rddms_service_viewers, group_names)
ok3, msg3 = check_group_membership(data_owners, group_names)
ok4, msg4 = check_group_membership(data_viewers, group_names)

if ok1 and ok2 and ok3 and ok4:
    detail_text = f"{data_owners},<br> {data_viewers}, <br>{rddms_service_owners} <br>{rddms_service_viewers}"
    green_banner("You are member of all required groups.", detail_text)
else:
    red_banner("You are NOT member of all required groups.")
    print(msg1)
    print(msg2)
    print(msg3)
    print(msg4)

if not created_tag_name:
    red_banner(f"Legal  tag'{created_tag_name}' does not exist.", "Please create it before proceeding.")
    SystemExit("Legal tag does not exist. Please create it before proceeding.")


legal_tag_name = created_tag_name
ok4, msg4 = check_legal_tag_exists(legal_tag_name)
if ok4:
    green_banner(f"Legal tag `{legal_tag_name}` exists.")
else:
    red_banner(f"Legal  tag'{legal_tag_name}' does not exist.", "Please create it before proceeding.")

### 3.2 Creating a new Dataspace

In [ ]:
# Define the new dataspace payload
user_dataspace_name = f"rddms_lab/Test_Dataspace_{user_id}"
new_dataspace_payload = [
    {
        "DataspaceId": user_dataspace_name,
        "Path": user_dataspace_name,
        "CustomData": {
            "legaltags": [f"{legal_tag_name}"],
            "otherRelevantDataCountries":["US"],
            "owners":[f"data.default.owners@{entitlements_domain}"],
            "viewers":[f"data.default.viewers@{entitlements_domain}"]
        }
    }
]
print(f"Creating new dataspace: {user_dataspace_name}")
body = json.dumps(new_dataspace_payload)

# Make the POST request to create the new dataspace
try:
    create_dataspace_response = requests.post(
        f"{RDDMS_REST_API}/dataspaces",
        headers=get_rddms_header(),
        data=body
    )
    if create_dataspace_response.status_code == 201:
        green_banner(f"Dataspace '{user_dataspace_name}' created successfully.")
    elif create_dataspace_response.status_code == 400:
        green_banner(f"Dataspace '{user_dataspace_name}' already exists.")
    else:
        red_banner(f"Failed to create dataspace. Status code: {create_dataspace_response.status_code}")
except requests.exceptions.RequestException as e:
    red_banner(f"Error creating dataspace: {e}")

encoded_dataspace_name = urllib.parse.quote(user_dataspace_name, safe="")

### 3.3 Search the Dataspace using OSDU Search API

In [ ]:
#search by legal tag
search_query = {
  "kind": "*:wks:dataset--ETPDataspace:*",
  "query": f"legal.legaltags: {legal_tag_name}",
  "returnedFields": [ "id", "data.DatasetProperties.URI", "legal" ],
  "trackTotalCount": True
}

try:
    results = requests.post(f"{search_endpoint}/query", headers=get_rddms_header(), json=search_query)
    if results.status_code == 200:
      if results.json().get('totalCount', 0) == 0:
        red_banner("No records found for the given legal tag.")
      else:
        first_result_id = results.json().get('results', [])[0].get('id', 'N/A')
        detail_text = f"{first_result_id}"
        green_banner(f"Found {results.json().get('totalCount', 0)} record(s) for the given legal tag.", detail_text)
      for item in results.json().get('results', []):
        print(f"Dataspace URI: {item.get('data', {}).get('DatasetProperties.URI', 'N/A')}")
    else:
      red_banner(f"Search query failed with status code: {results.status_code}")
except Exception as e:
    red_banner(f"Error during search query: {e}")

### 3.4 Deleting the Dataspace

This part will be covered in the last "Cleanup for Lab 3" section since we are going to use the newly generated Dataspace in the following sections.

By deferring the deletion to the cleanup phase, we ensure that the Dataspace remains available for all subsequent operations and tests, avoiding any interruptions or errors caused by premature deletion.

## 4. Writting Data to RDDMS

### 4.1 Starting a Transaction

To ensure data integrity, the RDDMS requires all write operations to be performed within a transaction.

This approach allows you to prepare the data for the dataspace and ensures that the data is safely pushed when the transaction is committed.

In [ ]:
response = requests.post(
    f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}/transactions",
    headers=get_rddms_header(),
)
if response.status_code == 201:
    rddms_transaction_id = response.text
    green_banner(f"Transaction created successfully in dataspace '{user_dataspace_name}'.")
else:
    red_banner(f"Failed to create transaction. Status code: {response.status_code}")

### 4.2 Pushing PointSet Meta-Data (JSON RESQML Object)

Upload a JSON RESQML object to the dataspace by using a transaction ID.

In [ ]:
# this contains a new Points object with 6 XYZ points with the associated CoordinateSystem definition
# The array value of XYZ points are stored in an external part.
payload = json.dumps([
  {
    "Citation": {
      "Title": "CustomTestCrs",
      "Originator": "dalsaab",
      "Creation": "2021-09-02T07:57:28.000Z",
      "Format": "Paradigm SKUA-GOCAD 22 Alpha 1 Build:20210830-0200 (id: origin/master|56050|1fb1cf919c2|20210827-1108) for Linux_x64_2.17_gcc91",
      "Editor": "dalsaab",
      "LastUpdate": "2021-09-06T13:30:24.000Z"
    },
    "YOffset": 6470000,
    "ZOffset": 0,
    "ArealRotation": {
      "_": 0,
      "$type": "eml20.PlaneAngleMeasure",
      "Uom": "rad"
    },
    "ProjectedAxisOrder": "easting northing",
    "ProjectedUom": "m",
    "VerticalUom": "m",
    "XOffset": 420000,
    "ZIncreasingDownward": True,
    "VerticalCrs": {
      "EpsgCode": 6230,
      "$type": "eml20.VerticalCrsEpsgCode"
    },
    "ProjectedCrs": {
      "EpsgCode": 23031,
      "$type": "eml20.ProjectedCrsEpsgCode"
    },
    "$type": "resqml20.obj_LocalDepth3dCrs",
    "SchemaVersion": "2.0",
    "Uuid": "7c7d7987-b7b9-4215-9014-cb7d6fb62173"
  },
  {
    "Citation": {
      "$type": "eml20.Citation",
      "Title": "Hdf Proxy",
      "Originator": "Mathieu",
      "Creation": "2014-09-09T15:33:25Z",
      "Format": "[F2I-CONSULTING:resqml2CppApi]"
    },
    "MimeType": "application/x-hdf5",
    "$type": "eml20.obj_EpcExternalPartReference",
    "SchemaVersion": "2.0.0.20140822",
    "Uuid": "68f2a7d4-f7c1-4a75-95e9-3c6a7029fb23"
  },
  {
    "Citation": {
      "Title": "Pointset 1",
      "Originator": "user1",
      "Creation": "2019-01-08T13:41:25.000Z",
      "Format": "Paradigm SKUA-GOCAD 22 Alpha 1 Build:20210830-0200 (id: origin/master|56050|1fb1cf919c2|20210827-1108) for Linux_x64_2.17_gcc91",
      "$type": "eml20.Citation"
    },
    "ExtraMetadata": [
      {
        "Name": "pdgm/dx/resqml/creatorGroup",
        "Value": "Interpreters",
        "$type": "resqml20.NameValuePair"
      }
    ],
    "NodePatch": [
      {
        "PatchIndex": 0,
        "Count": 6,
        "Geometry": {
          "$type": "resqml20.PointGeometry",
          "LocalCrs": {
            "$type": "eml20.DataObjectReference",
            "ContentType": "application/x-resqml+xml;version=2.0;type=obj_LocalDepth3dCrs",
            "Title": "CustomTestCrs",
            "UUID": "7c7d7987-b7b9-4215-9014-cb7d6fb62173"
          },
          "Points": {
            "$type": "resqml20.Point3dHdf5Array",
            "Coordinates": {
              "$type": "eml20.Hdf5Dataset",
              "PathInHdfFile": "/RESQML/5d27775e-5c7f-4786-a048-9a303fa1165a/points_patch0",
              "HdfProxy": {
                "$type": "eml20.DataObjectReference",
                "ContentType": "application/x-resqml+xml;version=2.0;type=obj_EpcExternalPartReference",
                "UUID": "68f2a7d4-f7c1-4a75-95e9-3c6a7029fb23",
                "DescriptionString": "Hdf Proxy",
                "VersionString": "1410276805"
              }
            }
          }
        }
      }
    ],
    "$type": "resqml20.obj_PointSetRepresentation",
    "SchemaVersion": "2.0.0.20140822",
    "Uuid": "5d27775e-5c7f-4786-a048-9a303fa1165a"
  }
])

response = requests.put(
    f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}/resources?transactionId={rddms_transaction_id}",
    headers=get_rddms_header(),
    data=payload
)
if response.status_code == 200:
    green_banner("Resource created successfully.")
else:
    red_banner(f"Failed to create resource. Status code: {response.status_code}")

### 4.3 Pushing the PoinSet Data Arrays containing the geometry

Upload the missing array data, which contains the XYZ points.

In [ ]:
# Now push the missing array in the External part containing the XYZ points
payload = json.dumps([
  {
    "ContainerType": "eml20.obj_EpcExternalPartReference",
    "ContainerUuid": "68f2a7d4-f7c1-4a75-95e9-3c6a7029fb23",
    "PathInResource": "/RESQML/5d27775e-5c7f-4786-a048-9a303fa1165a/points_patch0",
    "Dimensions": [
      3, 6
    ],
    "PreferredSubarrayDimensions": [
      3, 1
    ],
    "Data": [
      0, 0, 0,
      1, 0, 0,
      0, 1, 2,
      1, 1, 2,
      1, 0, 2,
      1, 1, 1
    ],
    "ArrayType": "Float32Array"
  }
])

response = requests.put(
    f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}/resources/arrays?transactionId={rddms_transaction_id}",
    headers=get_rddms_header(),
    data=payload
)
if response.status_code == 200 :
    green_banner("Array created successfully.")
else:
    red_banner(f"Failed to create array. Status code: {response.status_code}")


### 4.4 Committing to Finalize the Transaction

When the commit operation is performed, the dataspace will be temporarily locked for editing.

This ensures that no data corruption occurs if two users attempt to push data simultaneously. Any subsequent transactions will be queued and executed only after the first transaction is finalized.

In [ ]:
response = requests.put(
    f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}/transactions/{rddms_transaction_id}",
    headers=get_rddms_header()
)
if response.status_code == 200:
    green_banner("Transaction committed successfully.")
else:
    red_banner(f"Failed to commit transaction. Status code: {response.status_code}")

### 4.5 Verifying Data Objects

Check that objects and activity have been successfully added to the dataspace.

In [ ]:
response = requests.get(
    f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}/resources",
    headers=get_rddms_header()
)

response_flat = pd.json_normalize(response.json()).to_dict(orient='records')
response_df = pd.DataFrame.from_dict(response_flat)
pretty_print_panda_frame(response_df, title=f"All resources in the dataspace \"{user_dataspace_name}\"", show_index=False)

### 4.6 Optionally Cancelling the Transaction

You can cancel the transaction at any time after it has been created.

When a transaction is cancelled, all temporary changes prepared during the transaction will be rolled back, and no changes will be submitted to the dataspace.

In [ ]:
response = requests.delete(
    f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}/transactions/{rddms_transaction_id}",
    headers=get_rddms_header()
)
if response.status_code == 200:
    green_banner(f"Transaction deleted successfully in dataspace '{user_dataspace_name}'.")
else:
    red_banner(f"Failed to delete transaction. Status code: {response.status_code}")

## 5. Using the OSDU ETP client docker image

### 5.1 Downloading the Docker image

In [ ]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # use udocker to run docker commands in Google Colab
    #
    !pip install --quiet udocker
    !adduser --disabled-password --gecos '' udockerrunner
    docker_cmd = "sudo -u udockerrunner udocker"

    # Pull the latest version of the ETP Client from the Open Group registry
    !{docker_cmd} pull --registry=https://community.opengroup.org:5555 osdu/platform/domain-data-mgmt-services/reservoir/open-etp-server/open-etp-sslclient-main
    !{docker_cmd} tag osdu/platform/domain-data-mgmt-services/reservoir/open-etp-server/open-etp-sslclient-main open-etp-sslclient
else:
    # use docker to run docker commands in Jupyter Notebook
    !pip install --quiet docker
    docker_cmd = "docker"

    # Pull the latest version of the ETP Client from the Open Group registry
    !{docker_cmd} pull --quiet community.opengroup.org:5555/osdu/platform/domain-data-mgmt-services/reservoir/open-etp-server/open-etp-sslclient-main
    !{docker_cmd} tag community.opengroup.org:5555/osdu/platform/domain-data-mgmt-services/reservoir/open-etp-server/open-etp-sslclient-main open-etp-sslclient

# Set the variables to be use for the ETP connection
etp_credentials = f"--server-url {RDDMS_ETP_API} --data-partition-id {osdu_data_partition_id} --auth bearer --jwt-token {get_token()}"
space_root_cmd = f"/bin/openETPServer space {etp_credentials}"

# List all dataspaces
#list_dataspace_cmd = f"{space_root_cmd} space --list"
#!{docker_cmd} run --rm  --entrypoint=sh open-etp-sslclient -c "{list_dataspace_cmd}"

green_banner("Docker image is ready to be used.")

In [ ]:
# List all dataspaces
list_dataspace_cmd = f"{space_root_cmd} space --list"
!{docker_cmd} run --rm  --entrypoint=sh open-etp-sslclient -c "{list_dataspace_cmd}"

### 5.2 Creating the Dataspace

In [ ]:
xdata_acl = {
    "legaltags": [f"{legal_tag_name}"],
    "otherRelevantDataCountries":["US"],
    "owners":[f"data.default.owners@{entitlements_domain}"],
    "viewers":[f"data.default.viewers@{entitlements_domain}"]
    }
xdata_acl_json = json.dumps(xdata_acl).replace('"', '\\"')
new_dataspace_cmd = f"{space_root_cmd} space --new -s {user_dataspace_name} --xdata '{xdata_acl_json}'"

!{docker_cmd} run --rm  --entrypoint=sh open-etp-sslclient -c "{new_dataspace_cmd}"

### 5.3 Pushing RESQML file to the dataspace

Download sample RESQML files from  OSDU gitlab

In [ ]:
# URL of the .epc file to download
dataset_repository = "https://community.opengroup.org/osdu/platform/domain-data-mgmt-services/reservoir/open-etp-server/-/raw/main/data/"
epc_file = "Volve_Demo_Faults_Depth.epc"
epc_file_url = f"{dataset_repository}{epc_file}"

# URL of the .hdf5 file to download
hdf5_file = "Volve_Demo_Faults_Depth.h5"
hdf5_file_url = f"{dataset_repository}{hdf5_file}"


# Function to download a file
def download_file(url, save_path):
    if os.path.exists(save_path):
        print(f"File already exists at {save_path}. Skipping download.")
        return
    response = requests.get(url, stream=True)
    if response.ok:
        with open(save_path, "wb") as file:
            file.write(response.content)
        print(f"File downloaded successfully to {save_path}")
    else:
        print(f"Failed to download file from {url}. Status code: {response.status_code}")

# Download both files
download_file(hdf5_file_url, hdf5_file)
download_file(epc_file_url, epc_file)

Use the OpenETP Client Executable to import the RESQML file

In [ ]:
# Import a RESQML file into a space using the Open ETP SSL client
import_resqml_cmd = f"{space_root_cmd} space -s {user_dataspace_name} --import-epc /data/{epc_file}"

# Mount the local directory containing the RESQML file to the Docker container
if IN_COLAB:
    !{docker_cmd} run --rm --volume=$(pwd):/data --entrypoint=sh open-etp-sslclient -c "{import_resqml_cmd}"
else:
    !{docker_cmd} run --rm -v .:/data --entrypoint=sh open-etp-sslclient -c "{import_resqml_cmd}"

### 5.4 Listing the content of the dataspace

In [ ]:
# Check Dataspace Contents
check_dataspace_cmd = f"{space_root_cmd} space -s {user_dataspace_name} --stats"
print()
print("-" * 80)
print(f"Checking contents of dataspace {user_dataspace_name}")
print("-" * 80)
!{docker_cmd} run --rm  --entrypoint=sh open-etp-sslclient -c "{check_dataspace_cmd}"

## 6. Manifest generation helper and Ingestion

## Cleanup for Lab 3

This section deletes the resources created during Lab 2 to keep the OSDU instance clean. It attempts to delete:
* The InterpretedHorizon record.
* The newly created Dataspace


**Deleting the newly created Dataspace**

In [ ]:
if 'user_dataspace_name' in locals() and user_dataspace_name:
    try:
        print(f"Deleting dataspace: {user_dataspace_name}")
        encoded_dataspace_name = urllib.parse.quote(user_dataspace_name, safe="")
        delete_dataspace_response = requests.delete(
            f"{RDDMS_REST_API}/dataspaces/{encoded_dataspace_name}",
            headers=get_rddms_header()
        )
        if delete_dataspace_response.status_code == 204:
            green_banner(f"Dataspace '{user_dataspace_name}' deleted successfully.")
        else:
            red_banner(f"Failed to delete dataspace. Status code: {delete_dataspace_response.status_code}")
            # print(f"Response: {delete_dataspace_response.text}")
    except requests.exceptions.RequestException as e:
        red_banner(f"Error deleting dataspace: {e}")
else:
    green_banner("Dataspace name not found.")

**Deleting the Legal tag**

In [ ]:
# delete the legal tag
if 'created_tag_name' in locals() and created_tag_name:
    try:
        delete_legal_tag_response = requests.delete(
            f"{legal_endpoint}/legaltags/{created_tag_name}",
            headers=get_rddms_header()
        )
        if delete_legal_tag_response.status_code == 204:
            green_banner(f"Legal tag '{created_tag_name}' deleted successfully.")
        else:
            msg = f"Error: {delete_legal_tag_response.status_code} - {delete_legal_tag_response.text}"
            red_banner(f"Failed to delete legal tag '{created_tag_name}'.", msg)
    except requests.exceptions.RequestException as e:
        red_banner(f"Error deleting legal tag: {e}")
else:
    green_banner("Legal tag name not found.")


**Deleting the locally downloaded RESQML files**

In [ ]:
if 'epc_file' in locals() and epc_file:
    try:
        os.remove(epc_file)
        os.remove(hdf5_file)
        green_banner(f"Files '{epc_file}' and '{hdf5_file}' deleted successfully.")
    except FileNotFoundError:
        green_banner(f"Files '{epc_file}' and '{hdf5_file}' not found.")
    except Exception as e:
        red_banner(f"Error deleting files: {e}")
else:
    green_banner("File names not found.")

**Deleting the docker images**

In [ ]:
if 'docker_cmd' in locals() and docker_cmd:
    try:
        print(f"Removing docker image: open-etp-sslclient")
        !{docker_cmd} rmi open-etp-sslclient
        !{docker_cmd} rmi osdu/platform/domain-data-mgmt-services/reservoir/open-etp-server/open-etp-sslclient-main:latest
        green_banner("Docker images removed successfully.")
    except Exception as e:
        red_banner(f"Error removing docker image: {e}")

    if IN_COLAB:
        # Remove the created user
        try:
            print(f"Removing user: udockerrunner")
            !deluser --remove-home udockerrunner
            green_banner("User 'udockerrunner' removed successfully.")
        except Exception as e:
            red_banner(f"Error removing user: {e}")
else:
    green_banner("Docker command not found.")